<a href="https://colab.research.google.com/github/yuuu125/Lunette/blob/main/AI_Assistent1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai==0.28.1 python-docx notion-client langdetect pydub
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg  # 确保安装必要的依赖

import os
import re
import json
import openai
import whisper
from docx import Document
from google.colab import files, userdata
from notion_client import Client
from langdetect import detect, LangDetectException
import datetime
import tempfile
import torch
from pydub import AudioSegment
import subprocess

try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    NOTION_TOKEN = userdata.get('NOTION_TOKEN')
    NOTION_DB_ID = userdata.get('NOTION_DB_ID')

    if not OPENAI_API_KEY:
        raise ValueError("OPENAI_API_KEY not set")
    if not NOTION_TOKEN:
        print("⚠️ Notion token missing - feature disabled")
    if not NOTION_DB_ID:
        print("⚠️ Notion DB ID missing - feature disabled")

    openai.api_key = OPENAI_API_KEY
    print("✅ OpenAI API key set")

except Exception as e:
    print(f"❌ Key retrieval failed: {str(e)}")

def get_audio_duration(audio_path):
    """使用ffmpeg获取精确音频时长（秒）"""
    try:
        result = subprocess.run(
            ["ffprobe", "-v", "error", "-show_entries", "format=duration",
             "-of", "default=noprint_wrappers=1:nokey=1", audio_path],
            capture_output=True, text=True
        )
        return float(result.stdout)
    except Exception as e:
        print(f"⚠️ 无法获取精确时长，使用估算值: {e}")
        # 估算：8KB/s 是常见音频比特率
        return max(30, os.path.getsize(audio_path) // 8000)

def transcribe_audio(audio_path, model_size="base"):
    """使用Whisper转录音频文件"""
    print(f"🔊 Starting transcription with Whisper ({model_size} model)...")

    try:
        # 检查GPU加速
        device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"💻 Using device: {device.upper()}")

        # 加载模型
        model = whisper.load_model(model_size, device=device)
        print(f"✅ Loaded Whisper {model_size} model")

        # 转录音频
        result = model.transcribe(
            audio_path,
            fp16=(device == "cuda"),
            verbose=True,
            task="transcribe"
        )

        transcription = result["text"]
        print(f"✅ Transcription complete! Characters: {len(transcription)}")
        return transcription

    except Exception as e:
        print(f"❌ Transcription failed: {str(e)}")
        raise

def test_notion_connection():
    """测试Notion连接是否有效"""
    try:
        # 初始化Notion客户端，显式配置客户端禁用代理
        notion = Client(
        auth=NOTION_TOKEN,
        client=httpx.Client(proxies=None)  # 显式禁用代理
        )
        notion.databases.retrieve(database_id=NOTION_DB_ID)
        print("✅ Notion connection verified")
        return True
    except Exception as e:
        print(f"❌ Notion connection failed: {str(e)}")
        return False

def clean_transcript(text):
    """Cleans raw transcript text"""
    text = re.sub(r'\d{1,2}:\d{2}:\d{2}', '', text)
    text = re.sub(r'Speaker\s*\d+:?', '', text)
    return re.sub(r'\n\s*\n', '\n\n', text).strip()

def segment_text(text):
    """Segments text into paragraphs"""
    return [p.strip() for p in text.split('\n\n') if p.strip()]

def handle_transcript_input():
    """Handles transcript input methods"""
    print("\n=== Handling Transcript Input ===")
    print("Choose input method:")
    print("1 - Upload text file (.txt or .docx)")
    print("2 - Paste text directly")
    print("3 - Upload audio file (transcribe with Whisper)")

    input_method = input("Your choice (1/2/3): ")
    transcript_text = ""

    # 文本文件上传
    if input_method == "1":
        uploaded = files.upload()
        if not uploaded:
            print("⚠️ No files uploaded, switching to paste")
            transcript_text = input("Paste meeting transcript: ")
        else:
            filename = list(uploaded.keys())[0]
            print(f"✅ Uploaded: {filename}")

            # 文本文件处理
            if filename.endswith('.txt'):
                transcript_text = uploaded[filename].decode('utf-8')

            # DOCX处理
            elif filename.endswith('.docx'):
                with tempfile.NamedTemporaryFile(delete=False, suffix='.docx') as tmp:
                    tmp.write(uploaded[filename])
                    tmp_path = tmp.name

                doc = Document(tmp_path)
                transcript_text = "\n".join([para.text for para in doc.paragraphs])
                os.unlink(tmp_path)
            else:
                raise ValueError("Unsupported file format")

    # 文本粘贴
    elif input_method == "2":
        transcript_text = input("Paste meeting transcript: ")

    # 音频文件处理
    elif input_method == "3":
        uploaded_audio = files.upload()
        if not uploaded_audio:
            print("⚠️ No audio files uploaded, switching to text paste")
            transcript_text = input("Paste meeting transcript: ")
        else:
            filename = list(uploaded_audio.keys())[0]
            print(f"✅ Uploaded audio: {filename}")

            # 创建临时文件
            with tempfile.NamedTemporaryFile(delete=False, suffix=os.path.splitext(filename)[1]) as tmp:
                tmp.write(uploaded_audio[filename])
                audio_path = tmp.name

            print("\n⚡ Select transcription speed:")
            print("1 - Fast (tiny model, fastest, lower accuracy)")
            print("2 - Balanced (base model, recommended)")
            print("3 - High Quality (small model, slower)")

            speed_choice = input("Your choice (1/2/3): ") or "2"
            model_map = {"1": "tiny", "2": "base", "3": "small"}
            model_size = model_map.get(speed_choice, "base")

            # 获取音频时长
            try:
                duration = get_audio_duration(audio_path)
                print(f"⏱ Audio duration: {duration//60:.0f}m {duration%60:.0f}s")

                # 时间估算
                time_estimates = {"tiny": 0.3, "base": 0.8, "small": 2.0}
                est_sec = duration * time_estimates[model_size]
                print(f"⏳ Estimated processing time: ~{est_sec//60:.0f}m {est_sec%60:.0f}s")
            except Exception as e:
                print(f"⚠️ Duration estimation failed: {e}")

            # 转录音频
            transcript_text = transcribe_audio(audio_path, model_size)

            # 清理临时文件
            os.unlink(audio_path)

    else:
        print("⚠️ Invalid option, defaulting to text paste")
        transcript_text = input("Paste meeting transcript: ")

    cleaned_text = clean_transcript(transcript_text)
    segments = segment_text(cleaned_text)

    print(f"📝 Processed text: {len(segments)} segments, {len(cleaned_text)} characters")
    return cleaned_text, segments

def analyze_with_gpt(text, language='en'):
    """Analyzes text with GPT API"""
    print("\n=== Analyzing with GPT ===")

    if not openai.api_key:
        print("❌ OpenAI API key missing")
        return {"error": "OpenAI API key not set", "fallback_used": True}, 0

    # Language mapping
    lang_map = {'zh': 'Chinese', 'es': 'Spanish', 'fr': 'French', 'en': 'English'}
    lang_name = lang_map.get(language[:2], 'English')

    # System prompt setup
    system_prompt = f"""
    You are a professional meeting analyst. Extract key information:
    - Respond in {lang_name}
    - Use this JSON format:
    {{
        "meeting_title": "Meeting Title",
        "participants": ["Attendee1", "Attendee2"],
        "summary": "Meeting summary",
        "action_items": [{{"task": "Task", "assignee": "Owner"}}],
        "key_points": {{
            "concerns": [],
            "decisions": [],
            "deadlines": [],
            "updates": []
        }},
        "meeting_type": "Meeting type",
        "platform": "Platform",
        "fallback_used": false
    }}

    Extraction rules:
    1. meeting_title: Extract from start/end or generate
    2. participants: Extract all attendees
    3. Focus on meeting start/end sections
    """

    user_prompt = f"Meeting transcript:\n{text[:10000]}"

    try:
        # GPT API call
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.3
        )

        content = response.choices[0].message['content']
        result = json.loads(content)
        tokens_used = response.usage['total_tokens']

        print(f"✅ GPT analysis complete! Tokens: {tokens_used}")
        print(f"Meeting title: {result.get('meeting_title', 'N/A')}")
        print(f"Participants: {len(result.get('participants', []))}")
        print(f"Meeting type: {result.get('meeting_type', 'N/A')}")
        print(f"Action items: {len(result.get('action_items', []))}")

        # Fallback for action items
        if not result.get('action_items'):
            result['fallback_used'] = True
            print("⚠️ No action items detected")

        return result, tokens_used

    except Exception as e:
        print(f"❌ GPT analysis failed: {str(e)}")
        return {
            "error": str(e),
            "fallback_used": True
        }, 0

def create_notion_entry(meeting_data):
    """Creates Notion database entry"""
    if not NOTION_TOKEN or not NOTION_DB_ID:
        print("⚠️ Notion config incomplete - skipping")
        return False

    print("\n=== Syncing to Notion ===")

    try:
        # 初始化Notion客户端，显式配置客户端禁用代理
        notion = Client(
        auth=NOTION_TOKEN,
        client=httpx.Client(proxies=None)  # 显式禁用代理
)

        # Prepare properties
        properties = {
            "Meeting Title": {"title": [{"text": {"content": meeting_data.get("meeting_title", "Untitled")}}]},
            "Participant": {"rich_text": [{"text": {"content": ", ".join(meeting_data.get("participants", ["Unknown"]))}}]},
            "Date & Duration": {"date": {"start": meeting_data.get("date", datetime.datetime.now().isoformat())}},
            "Meeting Type": {"rich_text": [{"text": {"content": meeting_data.get("meeting_type", "Other")}}]},
            "Platform": {"select": {"name": meeting_data.get("platform", "Unknown")}},
            "Summary": {"rich_text": [{"text": {"content": meeting_data.get("summary", "")}}]},
            "Key Points": {"rich_text": [{"text": {"content": format_key_points(meeting_data)}}]},
            "Action Items": {"rich_text": [{"text": {"content": format_action_items(meeting_data)}}]},
        }

        # Create entry
        new_page = notion.pages.create(
            parent={"database_id": NOTION_DB_ID},
            properties=properties
        )

        print(f"✅ Notion entry created! ID: {new_page['id']}")
        return True
    except Exception as e:
        print(f"❌ Notion sync failed: {str(e)}")
        return False

def format_key_points(data):
    """Formats key points for Notion"""
    points = []
    key_points = data.get("key_points", {})
    for category, items in key_points.items():
        if items and isinstance(items, list):
            points.append(f"{category.upper()}:")
            points.extend([f"- {item}" for item in items])
    return "\n".join(points)

def format_action_items(data):
    """Formats action items for Notion"""
    action_items = data.get("action_items", [])
    if not action_items or not isinstance(action_items, list):
        return "No action items"

    formatted = []
    for item in action_items:
        if isinstance(item, dict):
            task = item.get('task', 'Unknown task')
            assignee = item.get('assignee', 'Unassigned')
            formatted.append(f"- {task} (Owner: {assignee})")
        else:
            formatted.append(f"- {str(item)}")
    return "\n".join(formatted)

def main():
    """Main workflow execution"""
    if not openai.api_key:
        print("❌ OpenAI API key missing")
        return

    logs = {"steps": [], "errors": []}

    # Test Notion connection
    if NOTION_TOKEN and NOTION_DB_ID:
        if not test_notion_connection():
            print("⚠️ Notion connection failed")

    try:
        # Process input
        cleaned_text, segments = handle_transcript_input()
        logs["steps"].append({
            "step": "Text input",
            "segment_count": len(segments),
            "status": "success"
        })

        # Detect language
        try:
            language = detect(cleaned_text[:500]) if cleaned_text else 'en'
        except LangDetectException:
            language = 'en'
        print(f"🌐 Detected language: {language}")

        # GPT analysis
        gpt_results, tokens_used = analyze_with_gpt(cleaned_text, language)

        if "error" in gpt_results:
            logs["steps"].append({
                "step": "GPT analysis",
                "status": "failed",
                "error": gpt_results["error"]
            })
            print(f"❌ GPT failed: {gpt_results['error']}")
            return
        else:
            logs["steps"].append({
                "step": "GPT analysis",
                "tokens_used": tokens_used,
                "meeting_title": gpt_results.get("meeting_title"),
                "participants_count": len(gpt_results.get("participants", [])),
                "meeting_type": gpt_results.get("meeting_type"),
                "action_items_count": len(gpt_results.get("action_items", [])),
                "status": "success"
            })

        # Add date and sync to Notion
        gpt_results["date"] = datetime.datetime.now().isoformat()
        notion_success = create_notion_entry(gpt_results)
        logs["steps"].append({
            "step": "Notion sync",
            "status": "success" if notion_success else "failed"
        })

        # Save logs
        with open("meeting_logs.json", "w") as f:
            json.dump(logs, f, indent=2)

        print("\n✅ Process complete! Logs saved")

    except Exception as e:
        logs["errors"].append(str(e))
        print(f"\n❌ Process error: {str(e)}")
        with open("error_log.json", "w") as f:
            json.dump(logs, f, indent=2)

if __name__ == "__main__":
    main()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 7.2 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.96.1
    Uninstalling openai-1.96.1:
      Successfully uninstalled openai-1.96.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.0.8 requires openai<2.0.0,>=1.10.0, but you have openai 0.28.1 which is incompatible.


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-484w9eyo
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-484w9eyo
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... canceled
ERROR: Operation cancelled by user
^C


In [ ]:
# 安装依赖
!pip uninstall -y langchain langchain-core langchain-community langchain-openai openai notion-client
!pip install langchain==0.2.0
!pip install langchain-core==0.2.38
!pip install langchain-community==0.2.0
!pip install langchain-openai==0.1.9
!pip install openai==1.37.0
!pip install notion-client==2.0.0
!pip install tqdm python-docx langdetect pydub httpx==0.27.0
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg -y

import os
import json
import re
import openai
import whisper
from docx import Document
from google.colab import files, userdata
from notion_client import Client, errors
from langdetect import detect, LangDetectException
import datetime
import tempfile
import torch
import subprocess
from tqdm import tqdm
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
import httpx

# 清除代理环境变量
for var in ['HTTP_PROXY', 'HTTPS_PROXY', 'http_proxy', 'https_proxy']:
    if var in os.environ:
        del os.environ[var]

# 初始化Notion客户端
notion = Client(
    auth=userdata.get('NOTION_TOKEN'),
    client=httpx.Client(proxies=None)
)

# ======================
# 初始化设置
# ======================
try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    NOTION_TOKEN = userdata.get('NOTION_TOKEN')
    NOTION_DB_ID = userdata.get('NOTION_DB_ID')
    NOTION_PAGE_ID = userdata.get('NOTION_PAGE_ID')

    missing_creds = []
    if not OPENAI_API_KEY:
        missing_creds.append("OPENAI_API_KEY")
    if not NOTION_TOKEN:
        missing_creds.append("NOTION_TOKEN")
    if not NOTION_DB_ID:
        missing_creds.append("NOTION_DB_ID")
    if not NOTION_PAGE_ID:
        missing_creds.append("NOTION_PAGE_ID")

    if missing_creds:
        raise ValueError(f"缺少凭证: {', '.join(missing_creds)}")

    print("✅ 所有凭证已设置")

except Exception as e:
    print(f"❌ 凭证获取失败: {str(e)}")
    print("\n🔧 设置说明:")
    print("1. 点击左侧边栏的钥匙图标（Colab密钥）")
    print("2. 添加以下密钥:")
    print("   - OPENAI_API_KEY: 你的OpenAI API密钥")
    print("   - NOTION_TOKEN: 你的Notion集成令牌")
    print("   - NOTION_DB_ID: Notion数据库ID")
    print("   - NOTION_PAGE_ID: 报告父页面ID")
    print("3. 添加后重新运行此单元格")
    raise

# ======================
# 日志系统
# ======================
class MeetingLogger:
    def __init__(self):
        self.logs = {
            "start_time": datetime.datetime.now().isoformat(),
            "steps": [],
            "errors": [],
            "metrics": {}
        }

    def log_step(self, step_name, status, details=None, error=None):
        entry = {
            "step": step_name,
            "timestamp": datetime.datetime.now().isoformat(),
            "status": status
        }
        if details:
            entry["details"] = details
        if error:
            entry["error"] = str(error)
        self.logs["steps"].append(entry)

    def log_metric(self, name, value):
        self.logs["metrics"][name] = value

    def save_logs(self, filename="meeting_logs.json"):
        with open(filename, "w") as f:
            json.dump(self.logs, f, indent=2)
        return filename

    def get_console_log(self):
        log_str = f"=== 会议处理日志 ===\n"
        log_str += f"开始时间: {self.logs['start_time']}\n"

        for step in self.logs["steps"]:
            status_icon = "✅" if step["status"] == "success" else "❌"
            log_str += f"{status_icon} [{step['timestamp']}] {step['step']}"
            if "details" in step:
                log_str += f" - {step['details']}"
            if step["status"] == "failed":
                log_str += f" - 错误: {step.get('error', '未知')}"
            log_str += "\n"

        if self.logs["metrics"]:
            log_str += "\n=== 指标 ===\n"
            for metric, value in self.logs["metrics"].items():
                log_str += f"- {metric}: {value}\n"

        return log_str

logger = MeetingLogger()

# ======================
# 工具函数：处理嵌套结构
# ======================
def flatten_key_points(key_points):
    """将key_points中的嵌套结构（字典/列表）转换为字符串，适配Notion格式"""
    flattened = {}
    for category, items in key_points.items():
        flattened_items = []
        for item in items:
            # 处理字典类型（如{"部门": ["问题1", "问题2"]}）
            if isinstance(item, dict):
                dict_strings = []
                for k, v in item.items():
                    # 字典的值如果是列表，转换为带符号的字符串
                    if isinstance(v, list):
                        list_str = "• ".join([str(i) for i in v])
                        dict_strings.append(f"{k}：• {list_str}")
                    else:
                        dict_strings.append(f"{k}：{v}")
                flattened_items.append("； ".join(dict_strings))

            # 处理列表类型（如["问题1", "问题2"]）
            elif isinstance(item, list):
                list_str = "• ".join([str(i) for i in item])
                flattened_items.append(f"• {list_str}")

            # 字符串直接保留
            else:
                flattened_items.append(str(item))
        flattened[category] = flattened_items
    return flattened

# ======================
# 音频处理
# ======================
def get_audio_duration(audio_path):
    try:
        result = subprocess.run(
            ["ffprobe", "-v", "error", "-show_entries", "format=duration",
             "-of", "default=noprint_wrappers=1:nokey=1", audio_path],
            capture_output=True, text=True
        )
        duration = float(result.stdout)
        logger.log_metric("音频时长(秒)", duration)
        return duration
    except Exception as e:
        logger.log_step("获取音频时长", "warning", error=e)
        return max(30, os.path.getsize(audio_path) // 8000)

def transcribe_audio(audio_path, model_size="base"):
    logger.log_step("音频转录", "started", {"模型大小": model_size, "音频路径": audio_path})

    try:
        device = "cuda" if torch.cuda.is_available() else "cpu"
        logger.log_step("硬件检查", "success", {"设备": device})

        model = whisper.load_model(model_size, device=device)
        logger.log_step("加载模型", "success")

        result = model.transcribe(
            audio_path,
            fp16=(device == "cuda"),
            verbose=False,
            task="transcribe"
        )

        transcription = result["text"]
        detected_lang = result["language"]
        logger.log_step("音频转录", "success", {
            "字符数": len(transcription),
            "检测语言": detected_lang
        })

        return transcription, detected_lang

    except Exception as e:
        logger.log_step("音频转录", "failed", error=e)
        raise

# ======================
# 会议分析模型与处理
# ======================
class MeetingAnalysis(BaseModel):
    meeting_title: str = Field(description="会议标题")
    participants: list[str] = Field(description="参与者名单")
    summary: str = Field(description="3-5段会议总结")
    key_points: dict = Field(description="按concerns、decisions、updates、risks分组的关键点（均为数组）")
    action_items: list[dict] = Field(description="行动项列表，包含task、assignee、due_date")
    meeting_type: str = Field(description="会议类型")
    platform: str = Field(description="会议平台")

def setup_langchain_chains(language='zh'):
    lang_map = {
        'zh': "用中文分析会议记录，输出严格符合JSON格式，key_points的子字段均为数组（用[]包裹）",
        'en': "Analyze the meeting transcript in English, output strict JSON with key_points as arrays",
        'fr': "Analyser le procès-verbal en français, sortie JSON stricte avec key_points en tableaux"
    }
    lang_instruction = lang_map.get(language[:2], lang_map['zh'])

    parser = JsonOutputParser(pydantic_object=MeetingAnalysis)

    prompt_template = PromptTemplate(
        template="""
        {lang_instruction}

        {format_instructions}

        ### 会议记录:
        {transcript}

        请严格按照格式要求输出，确保JSON结构正确。
        """,
        input_variables=["transcript"],
        partial_variables={
            "lang_instruction": lang_instruction,
            "format_instructions": parser.get_format_instructions()
        }
    )

    llm = ChatOpenAI(
        openai_api_key=OPENAI_API_KEY,
        temperature=0.3,
        model="gpt-3.5-turbo"
    )

    # 使用新的链式结构
    analysis_chain = prompt_template | llm | parser

    return analysis_chain

def analyze_meeting(transcript, language='zh'):
    logger.log_step("分析会议", "started", {"语言": language})
    print("\n开始分析会议内容...")

    try:
        analysis_chain = setup_langchain_chains(language)
        processed_transcript = transcript[:15000]
        print(f"使用的转录文本长度: {len(processed_transcript)}字符")

        parsed = analysis_chain.invoke({"transcript": processed_transcript})

        parsed["language"] = language
        parsed["date"] = datetime.datetime.now().isoformat()

        if not parsed.get("action_items"):
            logger.log_step("检查行动项", "warning", "未检测到行动项")
            print("⚠️ 未检测到行动项")
            parsed["fallback_used"] = True
        else:
            parsed["fallback_used"] = False

        logger.log_step("分析会议", "success", {
            "标题": parsed["meeting_title"],
            "参与者数量": len(parsed["participants"]),
            "行动项数量": len(parsed["action_items"])
        })
        print(f"✅ 会议分析完成 (标题: {parsed['meeting_title']})")
        return parsed

    except Exception as e:
        error_msg = f"分析会议失败: {str(e)}"
        logger.log_step("分析会议", "failed", error=error_msg)
        print(f"❌ {error_msg}")
        return {"error": error_msg, "fallback_used": True}

# ======================
# Notion报告生成
# ======================
def create_notion_report_page(meeting_data, transcript, logs):
    logger.log_step("创建Notion报告", "started")

    try:
        global notion

        # 验证父页面
        try:
            parent_page = notion.pages.retrieve(NOTION_PAGE_ID)
            page_title = parent_page.get('properties', {}).get('title', {}).get('title', [{}])[0].get('plain_text', '无标题')
            logger.log_step("父页面检查", "success", {"页面ID": NOTION_PAGE_ID, "标题": page_title})
            print(f"✅ 成功访问父页面: {page_title} (ID: {NOTION_PAGE_ID[:8]}...)")
        except errors.APIResponseError as e:
            if e.status == 404:
                error_msg = f"父页面不存在 (ID: {NOTION_PAGE_ID})。请检查ID是否正确。"
            elif e.status == 403:
                error_msg = f"没有访问父页面的权限 (ID: {NOTION_PAGE_ID})。请将页面共享给Notion集成。"
            else:
                error_msg = f"访问父页面失败: {str(e)}"
            logger.log_step("父页面检查", "failed", error=error_msg)
            print(f"❌ {error_msg}")
            return None
        except Exception as e:
            error_msg = f"父页面检查出错: {str(e)}"
            logger.log_step("父页面检查", "failed", error=error_msg)
            print(f"❌ {error_msg}")
            return None

        # 创建子页面
        new_page = notion.pages.create(
            parent={"page_id": NOTION_PAGE_ID},
            properties={
                "title": {
                    "title": [
                        {
                            "text": {
                                "content": meeting_data.get("meeting_title", "会议报告")[:200]
                            }
                        }
                    ]
                }
            }
        )
        page_id = new_page["id"]
        logger.log_step("创建子页面", "success", {"页面ID": page_id})
        print(f"✅ 已创建子页面 (ID: {page_id[:8]}...)")

        # 构建报告内容
        children_blocks = []

        # 1. 会议详情
        children_blocks.append({
            "object": "block",
            "type": "heading_2",
            "heading_2": {"rich_text": [{"text": {"content": "会议详情"}}]}
        })

        details_text = f"""
        **日期**: {meeting_data.get('date', '未知')}
        **参与者**: {', '.join(meeting_data.get('participants', []))}
        **语言**: {meeting_data.get('language', '未知')}
        **平台**: {meeting_data.get('platform', '未知')}
        **会议类型**: {meeting_data.get('meeting_type', '未知')}
        """
        children_blocks.append({
            "object": "block",
            "type": "paragraph",
            "paragraph": {"rich_text": [{"text": {"content": details_text.strip()}}]}
        })

        # 2. 会议总结
        children_blocks.append({
            "object": "block",
            "type": "heading_2",
            "heading_2": {"rich_text": [{"text": {"content": "总结"}}]}
        })
        children_blocks.append({
            "object": "block",
            "type": "paragraph",
            "paragraph": {"rich_text": [{"text": {"content": meeting_data.get('summary', '')}}]}
        })

        # 3. 关键点（修复嵌套结构问题）
        children_blocks.append({
            "object": "block",
            "type": "heading_2",
            "heading_2": {"rich_text": [{"text": {"content": "关键点"}}]}
        })

        key_points = meeting_data.get('key_points', {})
        key_points = flatten_key_points(key_points)

        for category, items in key_points.items():
            children_blocks.append({
                "object": "block",
                "type": "heading_3",
                "heading_3": {"rich_text": [{"text": {"content": category.capitalize()}}]}
            })

            if items:
                for item in items:
                    children_blocks.append({
                        "object": "block",
                        "type": "bulleted_list_item",
                        "bulleted_list_item": {"rich_text": [{"text": {"content": item}}]}
                    })

        # 4. 行动项
        children_blocks.append({
            "object": "block",
            "type": "heading_2",
            "heading_2": {"rich_text": [{"text": {"content": "行动项"}}]}
        })

        table_rows = []
        for idx, item in enumerate(meeting_data.get('action_items', [])):
            task = item.get('task', '')
            assignee = item.get('assignee', '未分配')
            due_date = item.get('due_date', '无')

            table_rows.append([
                [{"text": {"content": str(idx+1)}}],
                [{"text": {"content": task}}],
                [{"text": {"content": assignee}}],
                [{"text": {"content": due_date}}]
            ])

        children_blocks.append({
            "object": "block",
            "type": "table",
            "table": {
                "table_width": 4,
                "has_column_header": True,
                "has_row_header": False,
                "children": [
                    {
                        "object": "block",
                        "type": "table_row",
                        "table_row": {
                            "cells": [
                                [{"text": {"content": "序号"}}],
                                [{"text": {"content": "任务"}}],
                                [{"text": {"content": "负责人"}}],
                                [{"text": {"content": "截止日期"}}]
                            ]
                        }
                    },
                    *[{
                        "object": "block",
                        "type": "table_row",
                        "table_row": {"cells": cells}
                    } for cells in table_rows]
                ]
            }
        })

        # 5. 处理日志
        children_blocks.append({
            "object": "block",
            "type": "heading_2",
            "heading_2": {"rich_text": [{"text": {"content": "处理日志"}}]}
        })
        children_blocks.append({
            "object": "block",
            "type": "code",
            "code": {
                "rich_text": [{"text": {"content": logger.get_console_log()}}],
                "language": "plain text"
            }
        })

        # 添加内容到页面
        notion.blocks.children.append(
            block_id=page_id,
            children=children_blocks
        )
        logger.log_step("添加内容到页面", "success")
        print(f"✅ 已添加内容到子页面")


        # 关联数据库（修改后）
        if NOTION_DB_ID:
            try:
                db = notion.databases.retrieve(NOTION_DB_ID)
                logger.log_step("数据库验证", "success", {"db_id": NOTION_DB_ID})

        # 手动指定你的关系属性名称
                relation_prop_name = "relation"

        # 验证属性
                if relation_prop_name not in db["properties"]:
                    raise ValueError(f"数据库中不存在名为「{relation_prop_name}」的属性")
                if db["properties"][relation_prop_name]["type"] != "relation":
                    raise ValueError(f"属性「{relation_prop_name}」不是关系类型")

        # 关联
                notion.pages.update(
            page_id=page_id,
            properties={
                relation_prop_name: {
                    "relation": [{"id": NOTION_DB_ID}]
                }
            }
        )
                logger.log_step("关联数据库", "success", {"使用的属性": relation_prop_name})
                print(f"✅ 已通过属性「{relation_prop_name}」关联到数据库")
            except Exception as e:
                logger.log_step("关联数据库", "warning", error=str(e))
                print(f"⚠️ 关联数据库失败: {str(e)}（不影响报告生成）")

        report_url = new_page.get("url", "")
        logger.log_step("生成Notion报告", "success", {"URL": report_url})
        return report_url

    except Exception as e:
        error_details = f"Notion API错误: {str(e)}"
        if hasattr(e, 'response') and hasattr(e.response, 'content'):
            error_details += f"\n响应: {e.response.content.decode('utf-8')}"
        logger.log_step("生成Notion报告", "failed", error=error_details)
        print(f"❌ Notion操作失败: {error_details}")
        return None

# ======================
# 权限测试函数
# ======================
def test_notion_permissions():
    print("\n=== 开始Notion权限测试 ===")
    print(f"使用的父页面ID: {NOTION_PAGE_ID[:8]}... (完整: {NOTION_PAGE_ID})")

    # 1. 测试集成令牌有效性
    try:
        user_info = notion.users.me()
        print(f"✅ 集成令牌有效 (所属工作空间: {user_info.get('workspace_name', '未知')})")
    except errors.UnauthorizedError:
        print(f"❌ 集成令牌无效 (NOTION_TOKEN错误)")
        return False
    except Exception as e:
        print(f"❌ 验证集成令牌时出错: {str(e)}")
        return False

    # 2. 测试父页面访问权限
    try:
        page = notion.pages.retrieve(NOTION_PAGE_ID)
        page_title = page.get('properties', {}).get('title', {}).get('title', [{}])[0].get('plain_text', '无标题')
        print(f"✅ 成功访问父页面: {page_title}")
    except errors.APIResponseError as e:
        if e.status == 404:
            print(f"❌ 父页面不存在 (ID错误或页面已删除)")
            return False
        elif e.status == 403:
            print(f"❌ 没有访问权限 (请将页面共享给集成)")
            return False
        else:
            print(f"❌ 访问页面时出错 (状态码: {e.status}): {str(e)}")
            return False
    except Exception as e:
        print(f"❌ 访问页面时发生未知错误: {str(e)}")
        return False

    # 3. 测试数据库访问权限
    try:
        if NOTION_DB_ID:
            db = notion.databases.retrieve(NOTION_DB_ID)
            print(f"✅ 成功访问数据库: {db.get('title', [{}])[0].get('plain_text', '无标题')}")
    except errors.APIResponseError as e:
        print(f"⚠️ 数据库访问警告: {str(e)}（仍可生成报告，但可能无法关联）")

    return True

# ======================
# 输入处理
# ======================
def handle_transcript_input():
    logger.log_step("处理输入", "started")

    print("\n=== 输入方式选择 ===")
    print("1: 上传音频文件 (.mp3/.wav/.m4a/.opus)")
    print("2: 上传文本文件 (.txt/.docx)")
    print("3: 直接粘贴文本")

    try:
        choice = input("请选择输入方式 (1/2/3): ").strip() or "1"
    except:
        choice = "1"

    if choice == "1":
        # 音频处理
        uploaded = files.upload()
        if not uploaded:
            logger.log_step("上传音频", "failed", "未上传任何文件")
            print("⚠️ 未上传文件，切换到文本输入")
            return handle_transcript_input()

        filename = next(iter(uploaded.keys()))
        logger.log_step("上传音频", "success", {"文件名": filename, "大小": len(uploaded[filename])})
        print(f"✅ 已上传音频文件: {filename}")

        ext = os.path.splitext(filename)[1].lower()
        supported_audio = ['.mp3', '.wav', '.m4a', '.opus']
        if ext not in supported_audio:
            error = f"不支持的音频格式: {ext} (支持: {', '.join(supported_audio)})"
            logger.log_step("处理音频", "failed", error=error)
            print(f"❌ {error}")
            raise ValueError(error)

        with tempfile.NamedTemporaryFile(delete=False, suffix=ext) as tmp:
            tmp.write(uploaded[filename])
            audio_path = tmp.name

        # 选择模型
        print("\n⚡ 选择转录模型:")
        print("1: 快速 (tiny, 低精度)")
        print("2: 平衡 (base, 推荐)")
        print("3: 高精度 (small, 较慢)")
        try:
            model_choice = input("请选择模型 (1/2/3): ").strip() or "2"
        except:
            model_choice = "2"
        model_map = {"1": "tiny", "2": "base", "3": "small"}
        model_size = model_map.get(model_choice, "base")
        print(f"使用模型: {model_size}")

        # 转录
        duration = get_audio_duration(audio_path)
        print(f"音频时长: {duration:.1f}秒，开始转录...")
        transcript, detected_lang = transcribe_audio(audio_path, model_size)
        os.unlink(audio_path)

        print(f"✅ 转录完成 (语言: {detected_lang})")
        return transcript, detected_lang

    elif choice == "2":
        # 文本文件
        uploaded = files.upload()
        if not uploaded:
            logger.log_step("上传文本", "failed", "未上传任何文件")
            print("⚠️ 未上传文件，切换到直接粘贴")
            return handle_transcript_input()

        filename = next(iter(uploaded.keys()))
        logger.log_step("上传文本", "success", {"文件名": filename})
        print(f"✅ 已上传文件: {filename}")

        try:
            if filename.endswith('.txt'):
                transcript = uploaded[filename].decode('utf-8')
            elif filename.endswith('.docx'):
                with tempfile.NamedTemporaryFile(delete=False, suffix='.docx') as tmp:
                    tmp.write(uploaded[filename])
                    doc = Document(tmp.name)
                    transcript = "\n".join(p.text for p in doc.paragraphs)
                    os.unlink(tmp.name)
            else:
                raise ValueError(f"不支持的文件格式: {filename} (支持: .txt, .docx)")

            # 检测语言
            lang = detect(transcript[:500]) if transcript else 'zh'
            logger.log_step("检测语言", "success", {"语言": lang})
            print(f"✅ 读取完成 (检测语言: {lang})")
            return transcript, lang
        except Exception as e:
            logger.log_step("处理文本文件", "failed", error=str(e))
            print(f"❌ 处理文件出错: {str(e)}")
            raise

    elif choice == "3":
        # 直接粘贴
        print("\n请粘贴会议记录 (粘贴后按Enter，输入空行结束):")
        lines = []
        while True:
            line = input()
            if not line:
                break
            lines.append(line)
        transcript = "\n".join(lines)

        if not transcript.strip():
            logger.log_step("输入文本", "failed", "未输入任何内容")
            print("⚠️ 未输入任何内容，重新选择输入方式")
            return handle_transcript_input()

        # 检测语言
        try:
            lang = detect(transcript[:500])
            logger.log_step("检测语言", "success", {"语言": lang})
            print(f"✅ 已输入文本 (检测语言: {lang})")
        except:
            lang = 'zh'
            logger.log_step("检测语言", "warning", "使用默认语言中文")
            print(f"✅ 已输入文本 (使用默认语言: 中文)")

        return transcript, lang

    else:
        logger.log_step("选择输入方式", "warning", "无效选择，使用默认音频输入")
        print("⚠️ 无效选择，默认使用音频输入")
        return handle_transcript_input()

# ======================
# 主函数
# ======================
def main():
    logger.log_step("工作流程", "started")
    print("=== 会议记录处理工具 ===")

    try:
        if not test_notion_permissions():
            print("\n❌ 权限测试未通过，请先解决上述问题")
            log_file = logger.save_logs("error_logs.json")
            print(f"错误日志已保存到: {log_file}")
            return

        transcript, language = handle_transcript_input()
        logger.log_metric("转录文本长度", len(transcript))

        meeting_data = analyze_meeting(transcript, language)
        if "error" in meeting_data:
            raise RuntimeError(f"分析失败: {meeting_data['error']}")

        print("\n开始创建Notion报告...")
        report_url = create_notion_report_page(meeting_data, transcript, logger.logs)

        if not report_url:
            raise RuntimeError("创建Notion报告失败")

        log_file = logger.save_logs()
        print(f"\n🎉 处理完成！")
        print(f"📄 会议报告URL: {report_url}")
        print(f"📋 日志文件: {log_file}")

        from IPython.display import HTML
        display(HTML(f'<a href="{report_url}" target="_blank">点击打开Notion报告</a>'))

    except Exception as e:
        logger.log_step("工作流程", "failed", error=str(e))
        log_file = logger.save_logs("error_logs.json")
        print(f"\n❌ 处理失败！")
        print(f"错误详情: {str(e)}")
        print(f"错误日志已保存到: {log_file}")

if __name__ == "__main__":
    main()

In [ ]:
# 安装依赖 - 修改部分
!pip uninstall -y whisper
!pip install faster-whisper
!pip install git+https://github.com/openai/whisper.git  # 保留原接口但使用faster-whisper后端
!sudo apt update && sudo apt install ffmpeg -y

import os
import json
import re
import openai
import whisper
from docx import Document
from google.colab import files, userdata
from notion_client import Client, errors
from langdetect import detect, LangDetectException
import datetime
import tempfile
import torch
import subprocess
from tqdm import tqdm
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
import httpx
from whisper.utils import get_writer  # 导入faster-whisper的辅助工具
# 清除代理环境变量
for var in ['HTTP_PROXY', 'HTTPS_PROXY', 'http_proxy', 'https_proxy']:
    if var in os.environ:
        del os.environ[var]
# 初始化Notion客户端
http_client = httpx.Client()
http_client.proxies = None  # 禁用代理

notion = Client(
    auth=userdata.get('NOTION_TOKEN'),
    client=http_client
)

# ======================
# 初始化设置
# ======================
try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    NOTION_TOKEN = userdata.get('NOTION_TOKEN')
    NOTION_DB_ID = userdata.get('NOTION_DB_ID')
    NOTION_PAGE_ID = userdata.get('NOTION_PAGE_ID')

    missing_creds = []
    if not OPENAI_API_KEY:
        missing_creds.append("OPENAI_API_KEY")
    if not NOTION_TOKEN:
        missing_creds.append("NOTION_TOKEN")
    if not NOTION_DB_ID:
        missing_creds.append("NOTION_DB_ID")
    if not NOTION_PAGE_ID:
        missing_creds.append("NOTION_PAGE_ID")

    if missing_creds:
        raise ValueError(f"缺少凭证: {', '.join(missing_creds)}")

    print("✅ 所有凭证已设置")

except Exception as e:
    print(f"❌ 凭证获取失败: {str(e)}")
    print("\n🔧 设置说明:")
    print("1. 点击左侧边栏的钥匙图标（Colab密钥）")
    print("2. 添加以下密钥:")
    print("   - OPENAI_API_KEY: 你的OpenAI API密钥")
    print("   - NOTION_TOKEN: 你的Notion集成令牌")
    print("   - NOTION_DB_ID: Notion数据库ID")
    print("   - NOTION_PAGE_ID: 报告父页面ID")
    print("3. 添加后重新运行此单元格")
    raise

# ======================
# 日志系统
# ======================
class MeetingLogger:
    def __init__(self):
        self.logs = {
            "start_time": datetime.datetime.now().isoformat(),
            "steps": [],
            "errors": [],
            "metrics": {}
        }

    def log_step(self, step_name, status, details=None, error=None):
        entry = {
            "step": step_name,
            "timestamp": datetime.datetime.now().isoformat(),
            "status": status
        }
        if details:
            entry["details"] = details
        if error:
            entry["error"] = str(error)
        self.logs["steps"].append(entry)

    def log_metric(self, name, value):
        self.logs["metrics"][name] = value

    def save_logs(self, filename="meeting_logs.json"):
        with open(filename, "w") as f:
            json.dump(self.logs, f, indent=2)
        return filename

    def get_console_log(self):
        log_str = f"=== 会议处理日志 ===\n"
        log_str += f"开始时间: {self.logs['start_time']}\n"

        for step in self.logs["steps"]:
            status_icon = "✅" if step["status"] == "success" else "❌"
            log_str += f"{status_icon} [{step['timestamp']}] {step['step']}"
            if "details" in step:
                log_str += f" - {step['details']}"
            if step["status"] == "failed":
                log_str += f" - 错误: {step.get('error', '未知')}"
            log_str += "\n"

        if self.logs["metrics"]:
            log_str += "\n=== 指标 ===\n"
            for metric, value in self.logs["metrics"].items():
                log_str += f"- {metric}: {value}\n"

        return log_str

logger = MeetingLogger()

# ======================
# 工具函数：处理嵌套结构
# ======================
def flatten_key_points(key_points):
    """将key_points中的嵌套结构（字典/列表）转换为字符串，适配Notion格式"""
    flattened = {}
    for category, items in key_points.items():
        flattened_items = []
        for item in items:
            # 处理字典类型（如{"部门": ["问题1", "问题2"]}）
            if isinstance(item, dict):
                dict_strings = []
                for k, v in item.items():
                    # 字典的值如果是列表，转换为带符号的字符串
                    if isinstance(v, list):
                        list_str = "• ".join([str(i) for i in v])
                        dict_strings.append(f"{k}：• {list_str}")
                    else:
                        dict_strings.append(f"{k}：{v}")
                flattened_items.append("； ".join(dict_strings))

            # 处理列表类型（如["问题1", "问题2"]）
            elif isinstance(item, list):
                list_str = "• ".join([str(i) for i in item])
                flattened_items.append(f"• {list_str}")

            # 字符串直接保留
            else:
                flattened_items.append(str(item))
        flattened[category] = flattened_items
    return flattened

# ======================
# 音频处理 - 修改部分
# ======================
def transcribe_audio(audio_path, model_size="base"):
    logger.log_step("音频转录", "started", {"模型大小": model_size, "音频路径": audio_path})

    try:
        # 使用faster-whisper进行转录
        from faster_whisper import WhisperModel

        # 根据GPU可用性选择计算设备
        device = "cuda" if torch.cuda.is_available() else "cpu"
        compute_type = "float16" if device == "cuda" else "int8"

        logger.log_step("加载faster-whisper模型", "info", {
            "设备": device,
            "计算类型": compute_type,
            "模型大小": model_size
        })

        # 加载模型 - 使用本地缓存避免重复下载
        model = WhisperModel(
            model_size,
            device=device,
            compute_type=compute_type,
            download_root="/content/models"  # 设置模型缓存目录
        )

        # 执行转录
        logger.log_step("开始转录", "processing")
        segments, info = model.transcribe(
            audio_path,
            beam_size=5,  # 平衡速度和准确度
            vad_filter=True,  # 启用语音活动检测
            word_timestamps=False  # 不需要单词级时间戳
        )

        # 检测语言
        detected_lang = info.language
        logger.log_step("语言检测", "success", {"语言": detected_lang})

        # 收集转录文本
        transcription = ""
        segment_list = []

        # 使用进度条显示转录过程
        with tqdm(total=info.duration, unit='sec', desc="转录进度") as pbar:
            for segment in segments:
                segment_list.append(segment.text)
                pbar.update(segment.end - pbar.n)

        transcription = " ".join(segment_list)

        logger.log_step("音频转录", "success", {
            "字符数": len(transcription),
            "检测语言": detected_lang,
            "音频时长": f"{info.duration:.2f}秒"
        })

        return transcription, detected_lang

    except Exception as e:
        logger.log_step("音频转录", "failed", error=e)
        # 回退到原始whisper
        logger.log_step("尝试使用原始whisper", "fallback")
        try:
            model = whisper.load_model(model_size)
            result = model.transcribe(audio_path)
            return result["text"], result["language"]
        except Exception as fallback_error:
            logger.log_step("原始whisper回退失败", "failed", error=fallback_error)
            raise RuntimeError(f"转录失败: {str(e)} | 回退失败: {str(fallback_error)}")
# ======================
# 会议分析模型与处理
# ======================
class MeetingAnalysis(BaseModel):
    meeting_title: str = Field(description="会议标题")
    participants: list[str] = Field(description="参与者名单")
    summary: str = Field(description="3-5段会议总结")
    key_points: dict = Field(description="按concerns、decisions、updates、risks分组的关键点（均为数组）")
    action_items: list[dict] = Field(description="行动项列表，包含task、assignee、due_date")
    meeting_type: str = Field(description="会议类型")
    platform: str = Field(description="会议平台")

def setup_langchain_chains(language='zh'):
    lang_map = {
        'zh': "用中文分析会议记录，输出严格符合JSON格式，key_points的子字段均为数组（用[]包裹）",
        'en': "Analyze the meeting transcript in English, output strict JSON with key_points as arrays",
        'fr': "Analyser le procès-verbal en français, sortie JSON stricte avec key_points en tableaux"
    }
    lang_instruction = lang_map.get(language[:2], lang_map['zh'])

    parser = JsonOutputParser(pydantic_object=MeetingAnalysis)

    prompt_template = PromptTemplate(
        template="""
        {lang_instruction}

        {format_instructions}

        ### 会议记录:
        {transcript}

        请严格按照格式要求输出，确保JSON结构正确。
        """,
        input_variables=["transcript"],
        partial_variables={
            "lang_instruction": lang_instruction,
            "format_instructions": parser.get_format_instructions()
        }
    )

    llm = ChatOpenAI(
        openai_api_key=OPENAI_API_KEY,
        temperature=0.3,
        model="gpt-3.5-turbo"
    )

    # 使用新的链式结构
    analysis_chain = prompt_template | llm | parser

    return analysis_chain

def analyze_meeting(transcript, language='zh'):
    logger.log_step("分析会议", "started", {"语言": language})
    print("\n开始分析会议内容...")

    try:
        analysis_chain = setup_langchain_chains(language)
        processed_transcript = transcript[:15000]
        print(f"使用的转录文本长度: {len(processed_transcript)}字符")

        parsed = analysis_chain.invoke({"transcript": processed_transcript})

        parsed["language"] = language
        parsed["date"] = datetime.datetime.now().isoformat()

        if not parsed.get("action_items"):
            logger.log_step("检查行动项", "warning", "未检测到行动项")
            print("⚠️ 未检测到行动项")
            parsed["fallback_used"] = True
        else:
            parsed["fallback_used"] = False

        logger.log_step("分析会议", "success", {
            "标题": parsed["meeting_title"],
            "参与者数量": len(parsed["participants"]),
            "行动项数量": len(parsed["action_items"])
        })
        print(f"✅ 会议分析完成 (标题: {parsed['meeting_title']})")
        return parsed

    except Exception as e:
        error_msg = f"分析会议失败: {str(e)}"
        logger.log_step("分析会议", "failed", error=error_msg)
        print(f"❌ {error_msg}")
        return {"error": error_msg, "fallback_used": True}

# ======================
# Notion报告生成
# ======================
def create_notion_report_page(meeting_data, transcript, logs):
    logger.log_step("创建Notion报告", "started")

    try:
        global notion

        # 验证父页面
        try:
            parent_page = notion.pages.retrieve(NOTION_PAGE_ID)
            page_title = parent_page.get('properties', {}).get('title', {}).get('title', [{}])[0].get('plain_text', '无标题')
            logger.log_step("父页面检查", "success", {"页面ID": NOTION_PAGE_ID, "标题": page_title})
            print(f"✅ 成功访问父页面: {page_title} (ID: {NOTION_PAGE_ID[:8]}...)")
        except errors.APIResponseError as e:
            if e.status == 404:
                error_msg = f"父页面不存在 (ID: {NOTION_PAGE_ID})。请检查ID是否正确。"
            elif e.status == 403:
                error_msg = f"没有访问父页面的权限 (ID: {NOTION_PAGE_ID})。请将页面共享给Notion集成。"
            else:
                error_msg = f"访问父页面失败: {str(e)}"
            logger.log_step("父页面检查", "failed", error=error_msg)
            print(f"❌ {error_msg}")
            return None
        except Exception as e:
            error_msg = f"父页面检查出错: {str(e)}"
            logger.log_step("父页面检查", "failed", error=error_msg)
            print(f"❌ {error_msg}")
            return None

        # 创建子页面
        new_page = notion.pages.create(
            parent={"page_id": NOTION_PAGE_ID},
            properties={
                "title": {
                    "title": [
                        {
                            "text": {
                                "content": meeting_data.get("meeting_title", "会议报告")[:200]
                            }
                        }
                    ]
                }
            }
        )
        page_id = new_page["id"]
        logger.log_step("创建子页面", "success", {"页面ID": page_id})
        print(f"✅ 已创建子页面 (ID: {page_id[:8]}...)")

        # 构建报告内容
        children_blocks = []

        # 1. 会议详情
        children_blocks.append({
            "object": "block",
            "type": "heading_2",
            "heading_2": {"rich_text": [{"text": {"content": "会议详情"}}]}
        })

        details_text = f"""
        **日期**: {meeting_data.get('date', '未知')}
        **参与者**: {', '.join(meeting_data.get('participants', []))}
        **语言**: {meeting_data.get('language', '未知')}
        **平台**: {meeting_data.get('platform', '未知')}
        **会议类型**: {meeting_data.get('meeting_type', '未知')}
        """
        children_blocks.append({
            "object": "block",
            "type": "paragraph",
            "paragraph": {"rich_text": [{"text": {"content": details_text.strip()}}]}
        })

        # 2. 会议总结
        children_blocks.append({
            "object": "block",
            "type": "heading_2",
            "heading_2": {"rich_text": [{"text": {"content": "总结"}}]}
        })
        children_blocks.append({
            "object": "block",
            "type": "paragraph",
            "paragraph": {"rich_text": [{"text": {"content": meeting_data.get('summary', '')}}]}
        })

        # 3. 关键点（修复嵌套结构问题）
        children_blocks.append({
            "object": "block",
            "type": "heading_2",
            "heading_2": {"rich_text": [{"text": {"content": "关键点"}}]}
        })

        key_points = meeting_data.get('key_points', {})
        key_points = flatten_key_points(key_points)

        for category, items in key_points.items():
            children_blocks.append({
                "object": "block",
                "type": "heading_3",
                "heading_3": {"rich_text": [{"text": {"content": category.capitalize()}}]}
            })

            if items:
                for item in items:
                    children_blocks.append({
                        "object": "block",
                        "type": "bulleted_list_item",
                        "bulleted_list_item": {"rich_text": [{"text": {"content": item}}]}
                    })

        # 4. 行动项
        children_blocks.append({
            "object": "block",
            "type": "heading_2",
            "heading_2": {"rich_text": [{"text": {"content": "行动项"}}]}
        })

        table_rows = []
        for idx, item in enumerate(meeting_data.get('action_items', [])):
            task = item.get('task', '')
            assignee = item.get('assignee', '未分配')
            due_date = item.get('due_date', '无')

            table_rows.append([
                [{"text": {"content": str(idx+1)}}],
                [{"text": {"content": task}}],
                [{"text": {"content": assignee}}],
                [{"text": {"content": due_date}}]
            ])

        children_blocks.append({
            "object": "block",
            "type": "table",
            "table": {
                "table_width": 4,
                "has_column_header": True,
                "has_row_header": False,
                "children": [
                    {
                        "object": "block",
                        "type": "table_row",
                        "table_row": {
                            "cells": [
                                [{"text": {"content": "序号"}}],
                                [{"text": {"content": "任务"}}],
                                [{"text": {"content": "负责人"}}],
                                [{"text": {"content": "截止日期"}}]
                            ]
                        }
                    },
                    *[{
                        "object": "block",
                        "type": "table_row",
                        "table_row": {"cells": cells}
                    } for cells in table_rows]
                ]
            }
        })

        # 5. 处理日志
        children_blocks.append({
            "object": "block",
            "type": "heading_2",
            "heading_2": {"rich_text": [{"text": {"content": "处理日志"}}]}
        })
        children_blocks.append({
            "object": "block",
            "type": "code",
            "code": {
                "rich_text": [{"text": {"content": logger.get_console_log()}}],
                "language": "plain text"
            }
        })

        # 添加内容到页面
        notion.blocks.children.append(
            block_id=page_id,
            children=children_blocks
        )
        logger.log_step("添加内容到页面", "success")
        print(f"✅ 已添加内容到子页面")


        # 关联数据库（修改后）
        if NOTION_DB_ID:
            try:
                db = notion.databases.retrieve(NOTION_DB_ID)
                logger.log_step("数据库验证", "success", {"db_id": NOTION_DB_ID})

        # 手动指定你的关系属性名称
                relation_prop_name = "relation"

        # 验证属性
                if relation_prop_name not in db["properties"]:
                    raise ValueError(f"数据库中不存在名为「{relation_prop_name}」的属性")
                if db["properties"][relation_prop_name]["type"] != "relation":
                    raise ValueError(f"属性「{relation_prop_name}」不是关系类型")

        # 关联
                notion.pages.update(
            page_id=page_id,
            properties={
                relation_prop_name: {
                    "relation": [{"id": NOTION_DB_ID}]
                }
            }
        )
                logger.log_step("关联数据库", "success", {"使用的属性": relation_prop_name})
                print(f"✅ 已通过属性「{relation_prop_name}」关联到数据库")
            except Exception as e:
                logger.log_step("关联数据库", "warning", error=str(e))
                print(f"⚠️ 关联数据库失败: {str(e)}（不影响报告生成）")

        report_url = new_page.get("url", "")
        logger.log_step("生成Notion报告", "success", {"URL": report_url})
        return report_url

    except Exception as e:
        error_details = f"Notion API错误: {str(e)}"
        if hasattr(e, 'response') and hasattr(e.response, 'content'):
            error_details += f"\n响应: {e.response.content.decode('utf-8')}"
        logger.log_step("生成Notion报告", "failed", error=error_details)
        print(f"❌ Notion操作失败: {error_details}")
        return None

# ======================
# 权限测试函数
# ======================
def test_notion_permissions():
    print("\n=== 开始Notion权限测试 ===")
    print(f"使用的父页面ID: {NOTION_PAGE_ID[:8]}... (完整: {NOTION_PAGE_ID})")

    # 1. 测试集成令牌有效性
    try:
        user_info = notion.users.me()
        print(f"✅ 集成令牌有效 (所属工作空间: {user_info.get('workspace_name', '未知')})")
    except errors.UnauthorizedError:
        print(f"❌ 集成令牌无效 (NOTION_TOKEN错误)")
        return False
    except Exception as e:
        print(f"❌ 验证集成令牌时出错: {str(e)}")
        return False

    # 2. 测试父页面访问权限
    try:
        page = notion.pages.retrieve(NOTION_PAGE_ID)
        page_title = page.get('properties', {}).get('title', {}).get('title', [{}])[0].get('plain_text', '无标题')
        print(f"✅ 成功访问父页面: {page_title}")
    except errors.APIResponseError as e:
        if e.status == 404:
            print(f"❌ 父页面不存在 (ID错误或页面已删除)")
            return False
        elif e.status == 403:
            print(f"❌ 没有访问权限 (请将页面共享给集成)")
            return False
        else:
            print(f"❌ 访问页面时出错 (状态码: {e.status}): {str(e)}")
            return False
    except Exception as e:
        print(f"❌ 访问页面时发生未知错误: {str(e)}")
        return False

    # 3. 测试数据库访问权限
    try:
        if NOTION_DB_ID:
            db = notion.databases.retrieve(NOTION_DB_ID)
            print(f"✅ 成功访问数据库: {db.get('title', [{}])[0].get('plain_text', '无标题')}")
    except errors.APIResponseError as e:
        print(f"⚠️ 数据库访问警告: {str(e)}（仍可生成报告，但可能无法关联）")

    return True

# ======================
# 输入处理
# ======================
# 在 handle_transcript_input 函数中添加缺失的音频上传代码
def handle_transcript_input():
    logger.log_step("处理输入", "started")

    print("\n=== 输入方式选择 ===")
    print("1: 上传音频文件 (.mp3/.wav/.m4a/.opus)")
    print("2: 上传文本文件 (.txt/.docx)")
    print("3: 直接粘贴文本")

    try:
        choice = input("请选择输入方式 (1/2/3): ").strip() or "1"
    except:
        choice = "1"

    if choice == "1":
        # === 添加的音频上传代码开始 ===
        print("\n请上传音频文件 (支持.mp3/.wav/.m4a/.opus):")
        uploaded = files.upload()
        if not uploaded:
            logger.log_step("上传音频", "failed", "未上传任何文件")
            print("⚠️ 未上传文件，请重新选择输入方式")
            return handle_transcript_input()

        filename = next(iter(uploaded.keys()))
        audio_ext = os.path.splitext(filename)[1].lower()
        if audio_ext not in ['.mp3', '.wav', '.m4a', '.opus']:
            logger.log_step("上传音频", "failed", f"不支持的文件格式: {filename}")
            print(f"❌ 不支持的文件格式: {filename} (请上传.mp3/.wav/.m4a/.opus)")
            return handle_transcript_input()

        # 保存上传的音频文件
        audio_path = f"/tmp/{filename}"
        with open(audio_path, 'wb') as f:
            f.write(uploaded[filename])
        logger.log_step("上传音频", "success", {"文件名": filename, "路径": audio_path})
        print(f"✅ 已上传音频: {filename}")
        # === 添加的音频上传代码结束 ===

        # 增强模型选择
        print("\n⚡ 选择转录模型 (使用faster-whisper):")
        print("1: 极速 (tiny, 最快但精度较低)")
        print("2: 快速 (base, 推荐日常使用)")
        print("3: 平衡 (small, 速度和精度平衡)")
        print("4: 高精度 (medium, 会议记录推荐)")
        print("5: 专业级 (large, 最高精度)")

        try:
            model_choice = input("请选择模型 (1-5): ").strip() or "4"
        except:
            model_choice = "4"

        model_map = {
            "1": "tiny",
            "2": "base",
            "3": "small",
            "4": "medium",
            "5": "large"
        }

        model_size = model_map.get(model_choice, "medium")
        print(f"使用模型: {model_size}")

        # 获取音频时长
        duration = get_audio_duration(audio_path)
        print(f"音频时长: {duration:.1f}秒，开始转录...")

        # 添加性能提示
        if duration > 600:  # 超过10分钟
            print("⏳ 较长音频处理中... 请耐心等待 (可使用Colab Pro获得GPU加速)")
        elif model_size in ["medium", "large"]:
            print("🔍 使用高精度模型，可能需要更长时间...")

        transcript, detected_lang = transcribe_audio(audio_path, model_size)
        os.unlink(audio_path)  # 删除临时文件

        print(f"✅ 转录完成 (语言: {detected_lang})")
        return transcript, detected_lang

    # ... [其他输入方式的代码保持不变] ...

    elif choice == "2":
        # 文本文件
        uploaded = files.upload()
        if not uploaded:
            logger.log_step("上传文本", "failed", "未上传任何文件")
            print("⚠️ 未上传文件，切换到直接粘贴")
            return handle_transcript_input()

        filename = next(iter(uploaded.keys()))
        logger.log_step("上传文本", "success", {"文件名": filename})
        print(f"✅ 已上传文件: {filename}")

        try:
            if filename.endswith('.txt'):
                transcript = uploaded[filename].decode('utf-8')
            elif filename.endswith('.docx'):
                with tempfile.NamedTemporaryFile(delete=False, suffix='.docx') as tmp:
                    tmp.write(uploaded[filename])
                    doc = Document(tmp.name)
                    transcript = "\n".join(p.text for p in doc.paragraphs)
                    os.unlink(tmp.name)
            else:
                raise ValueError(f"不支持的文件格式: {filename} (支持: .txt, .docx)")

            # 检测语言
            lang = detect(transcript[:500]) if transcript else 'zh'
            logger.log_step("检测语言", "success", {"语言": lang})
            print(f"✅ 读取完成 (检测语言: {lang})")
            return transcript, lang
        except Exception as e:
            logger.log_step("处理文本文件", "failed", error=str(e))
            print(f"❌ 处理文件出错: {str(e)}")
            raise

    elif choice == "3":
        # 直接粘贴
        print("\n请粘贴会议记录 (粘贴后按Enter，输入空行结束):")
        lines = []
        while True:
            line = input()
            if not line:
                break
            lines.append(line)
        transcript = "\n".join(lines)

        if not transcript.strip():
            logger.log_step("输入文本", "failed", "未输入任何内容")
            print("⚠️ 未输入任何内容，重新选择输入方式")
            return handle_transcript_input()

        # 检测语言
        try:
            lang = detect(transcript[:500])
            logger.log_step("检测语言", "success", {"语言": lang})
            print(f"✅ 已输入文本 (检测语言: {lang})")
        except:
            lang = 'zh'
            logger.log_step("检测语言", "warning", "使用默认语言中文")
            print(f"✅ 已输入文本 (使用默认语言: 中文)")

        return transcript, lang

    else:
        logger.log_step("选择输入方式", "warning", "无效选择，使用默认音频输入")
        print("⚠️ 无效选择，默认使用音频输入")
        return handle_transcript_input()

# ======================
# 主函数
# ======================
def main():
    logger.log_step("工作流程", "started")
    print("=== 会议记录处理工具 ===")

    try:
        if not test_notion_permissions():
            print("\n❌ 权限测试未通过，请先解决上述问题")
            log_file = logger.save_logs("error_logs.json")
            print(f"错误日志已保存到: {log_file}")
            return

        transcript, language = handle_transcript_input()
        logger.log_metric("转录文本长度", len(transcript))

        meeting_data = analyze_meeting(transcript, language)
        if "error" in meeting_data:
            raise RuntimeError(f"分析失败: {meeting_data['error']}")

        print("\n开始创建Notion报告...")
        report_url = create_notion_report_page(meeting_data, transcript, logger.logs)

        if not report_url:
            raise RuntimeError("创建Notion报告失败")

        log_file = logger.save_logs()
        print(f"\n🎉 处理完成！")
        print(f"📄 会议报告URL: {report_url}")
        print(f"📋 日志文件: {log_file}")

        from IPython.display import HTML
        display(HTML(f'<a href="{report_url}" target="_blank">点击打开Notion报告</a>'))

    except Exception as e:
        logger.log_step("工作流程", "failed", error=str(e))
        log_file = logger.save_logs("error_logs.json")
        print(f"\n❌ 处理失败！")
        print(f"错误详情: {str(e)}")
        print(f"错误日志已保存到: {log_file}")

if __name__ == "__main__":
    main()




In [ ]:
# 安装必要依赖（指定兼容版本）
!pip uninstall -y whisper
!pip install faster-whisper==0.10.0  # 锁定版本以避免API变更
!pip install git+https://github.com/openai/whisper.git
!pip install tqdm python-docx notion-client langdetect langchain==0.1.13 langchain-openai==0.0.8 pydantic==2.5.2 httpx==0.27.0
!sudo apt update && sudo apt install ffmpeg -y

import os
import json
import re
import torch
import subprocess
import datetime
import tempfile
from tqdm import tqdm
from pydantic import BaseModel, Field
import httpx
import concurrent.futures
from functools import partial

# 导入第三方库
from google.colab import files, userdata
from notion_client import Client, errors
from langdetect import detect, LangDetectException
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser

# 清除代理环境变量（避免网络连接问题）
for var in ['HTTP_PROXY', 'HTTPS_PROXY', 'http_proxy', 'https_proxy']:
    if var in os.environ:
        del os.environ[var]

# 初始化Notion客户端（修复httpx代理参数问题）
http_client = httpx.Client()
http_client.proxies = None  # 禁用代理

notion = Client(
    auth=userdata.get('NOTION_TOKEN'),
    client=http_client
)

# ======================
# 配置参数与初始化
# ======================
try:
    # 从环境变量获取密钥
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    NOTION_TOKEN = userdata.get('NOTION_TOKEN')
    NOTION_PAGE_ID = userdata.get('NOTION_PAGE_ID')

    # 长音频处理参数
    CHUNK_DURATION = 900  # 每块15分钟（秒）
    OVERLAP_DURATION = 30  # 块间重叠30秒
    MAX_CONCURRENT_CHUNKS = 1  # 单线程处理，避免文件竞争
    # Notion块数量限制相关参数
    MAX_TRANSCRIPT_SEGMENTS = 50  # 最多显示50条转录文本（避免块数量超限）

    # 验证必要密钥
    missing_creds = []
    if not OPENAI_API_KEY:
        missing_creds.append("OPENAI_API_KEY")
    if not NOTION_TOKEN:
        missing_creds.append("NOTION_TOKEN")
    if not NOTION_PAGE_ID:
        missing_creds.append("NOTION_PAGE_ID")

    if missing_creds:
        raise ValueError(f"缺少必要凭证: {', '.join(missing_creds)}")

    print("✅ 所有凭证已配置完成，准备处理会议内容")

except Exception as e:
    print(f"❌ 初始化失败: {str(e)}")
    print("\n设置指南:")
    print("1. 点击左侧边栏的钥匙图标（Colab Secrets）")
    print("2. 添加以下密钥:")
    print("   - OPENAI_API_KEY: 你的OpenAI API密钥")
    print("   - NOTION_TOKEN: Notion集成令牌")
    print("   - NOTION_PAGE_ID: 用于存储报告的Notion页面ID")
    raise

# ======================
# 日志记录系统
# ======================
class MeetingLogger:
    def __init__(self):
        self.logs = {
            "start_time": datetime.datetime.now().isoformat(),
            "steps": [],
            "chunk_status": {}
        }

    def log_step(self, step_name, status, details=None, error=None):
        """记录处理步骤"""
        entry = {
            "step": step_name,
            "timestamp": datetime.datetime.now().isoformat(),
            "status": status
        }
        if details:
            entry["details"] = details
        if error:
            entry["error"] = str(error)
        self.logs["steps"].append(entry)

    def log_chunk(self, chunk_id, status, error=None):
        """记录分块处理状态"""
        self.logs["chunk_status"][chunk_id] = {
            "status": status,
            "timestamp": datetime.datetime.now().isoformat(),
            "error": str(error) if error else None
        }

    def get_completed_chunks(self):
        """获取成功的分块ID"""
        return [k for k, v in self.logs["chunk_status"].items() if v["status"] == "success"]

    def get_failed_chunks(self):
        """获取失败的分块ID"""
        return [k for k, v in self.logs["chunk_status"].items() if v["status"] == "failed"]

    def save_logs(self, filename="meeting_logs.json"):
        """保存日志到文件"""
        with open(filename, "w") as f:
            json.dump(self.logs, f, indent=2)
        return filename

# 初始化日志系统
logger = MeetingLogger()

# ======================
# 音频处理工具
# ======================
def get_audio_duration(audio_path):
    """获取音频时长（秒）"""
    try:
        result = subprocess.run(
            ["ffmpeg", "-i", audio_path],
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True
        )
        output = result.stdout

        duration_match = re.search(r"Duration: (\d+:\d+:\d+\.\d+)", output)
        if not duration_match:
            return 0.0

        duration_str = duration_match.group(1)
        h, m, s = duration_str.split(':')
        return float(h) * 3600 + float(m) * 60 + float(s)

    except Exception as e:
        logger.log_step("获取音频时长", "warning", error=str(e))
        return 0.0

def split_audio_into_chunks(audio_path):
    """将长音频分割为带重叠的块"""
    logger.log_step("音频分块", "started")

    try:
        total_duration = get_audio_duration(audio_path)
        if total_duration <= 0:
            raise ValueError("无法获取有效音频时长，可能文件损坏")

        # 计算分块数量
        num_chunks = max(1, int((total_duration + CHUNK_DURATION - OVERLAP_DURATION) //
                              (CHUNK_DURATION - OVERLAP_DURATION)))
        logger.log_step("计算分块数量", "success", {"总时长(分钟)": f"{total_duration/60:.1f}", "分块数": num_chunks})
        print(f"📊 音频将分割为 {num_chunks} 块（每块15分钟，重叠30秒）")

        # 创建临时目录存储分块
        chunk_dir = tempfile.mkdtemp()
        chunks = []

        for i in range(num_chunks):
            start_time = i * (CHUNK_DURATION - OVERLAP_DURATION)
            end_time = min(start_time + CHUNK_DURATION, total_duration)

            # 格式化为ffmpeg时间格式
            start_str = str(datetime.timedelta(seconds=start_time))
            duration_str = str(datetime.timedelta(seconds=end_time - start_time))

            chunk_path = f"{chunk_dir}/chunk_{i:03d}.wav"

            # 使用ffmpeg切割音频
            subprocess.run(
                [
                    "ffmpeg", "-y",  # 覆盖现有文件
                    "-i", audio_path,
                    "-ss", start_str,  # 起始时间
                    "-t", duration_str,  # 持续时间
                    "-ar", "16000",  # 统一采样率
                    "-ac", "1",  # 单声道
                    "-acodec", "pcm_s16le",  # 无损编码
                    chunk_path
                ],
                check=True,
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE
            )

            # 验证分块文件
            if not os.path.exists(chunk_path) or os.path.getsize(chunk_path) < 1024:
                raise RuntimeError(f"分块 {i} 生成失败，文件大小异常")

            chunks.append({
                "id": i,
                "path": chunk_path,
                "start_time": start_time,
                "end_time": end_time,
                "dir": chunk_dir  # 保留目录引用，避免提前删除
            })

        logger.log_step("音频分块", "success")
        return chunks

    except Exception as e:
        logger.log_step("音频分块", "failed", error=str(e))
        raise RuntimeError(f"音频分块失败: {str(e)}")

# ======================
# 语音转录（核心修复部分）
# ======================
# 全局模型变量（确保单例）
global_model = None

def transcribe_chunk(chunk):
    """转录单个音频块（修复Segment属性问题）"""
    chunk_id = chunk["id"]
    chunk_path = chunk["path"]
    start_time = chunk["start_time"]

    try:
        from faster_whisper import WhisperModel
        global global_model

        # 初始化模型（单例模式）
        if global_model is None:
            if not torch.cuda.is_available():
                raise RuntimeError("请切换到GPU环境（Runtime > Change runtime type）")

            print(f"🔧 加载 {chunk['model_size']} 模型（首次运行需下载约1.5GB）...")
            global_model = WhisperModel(
                chunk["model_size"],
                device="cuda",
                compute_type="float16",
                download_root="/content/models"
            )
            print(f"✅ 模型加载成功")

        # 验证文件可访问
        if not os.path.exists(chunk_path):
            raise FileNotFoundError(f"分块文件不存在: {chunk_path}")

        # 执行转录（不依赖confidence属性）
        segments, info = global_model.transcribe(
            chunk_path,
            beam_size=2,
            vad_filter=True,  # 启用语音活动检测
            vad_parameters=dict(min_silence_duration_ms=300),
            language=None  # 自动检测语言
        )

        # 处理转录结果（移除confidence属性）
        timestamped_segments = []
        for seg in segments:
            timestamped_segments.append({
                "text": seg.text.strip(),
                "start": seg.start + start_time,  # 转换为全局时间
                "end": seg.end + start_time
                # 移除confidence属性，兼容新版本API
            })

        logger.log_chunk(chunk_id, "success")
        return {
            "chunk_id": chunk_id,
            "segments": timestamped_segments,
            "language": info.language
        }

    except Exception as e:
        logger.log_chunk(chunk_id, "failed", error=str(e))
        print(f"⚠️ 分块 {chunk_id} 转录失败: {str(e)}")
        return None

def transcribe_long_audio(audio_path, model_size="small.en"):
    """转录长音频（分块处理）"""
    logger.log_step("音频转录", "started", {"模型": model_size})

    try:
        global global_model
        global_model = None  # 重置模型

        # 分割音频为块
        chunks = split_audio_into_chunks(audio_path)
        num_chunks = len(chunks)

        # 为每个分块添加模型参数
        for chunk in chunks:
            chunk["model_size"] = model_size

        # 转录所有分块（单线程避免文件竞争）
        print(f"🚀 开始转录 {num_chunks} 个分块...")
        results = []
        for chunk in tqdm(chunks, desc="转录进度"):
            result = transcribe_chunk(chunk)
            if result:
                results.append(result)

        # 重试失败的分块
        failed_ids = logger.get_failed_chunks()
        if failed_ids:
            print(f"🔄 重试 {len(failed_ids)} 个失败分块...")
            for chunk in chunks:
                if chunk["id"] in failed_ids:
                    result = transcribe_chunk(chunk)
                    if result:
                        results.append(result)

        # 按分块ID排序并合并结果
        results.sort(key=lambda x: x["chunk_id"])
        all_segments = []
        for res in results:
            all_segments.extend(res["segments"])

        # 清理临时文件（最后清理，避免读取失败）
        unique_dirs = list({chunk["dir"] for chunk in chunks})
        for dir_path in unique_dirs:
            if os.path.exists(dir_path):
                for f in os.listdir(dir_path):
                    os.unlink(os.path.join(dir_path, f))
                os.rmdir(dir_path)

        # 检测语言
        language = results[0]["language"] if results else "en"

        # 验证转录结果
        if not all_segments:
            raise RuntimeError("未获取到有效转录内容，请检查音频质量")

        logger.log_step("音频转录", "success", {"总片段数": len(all_segments)})
        print(f"✅ 转录完成（{len(all_segments)}个片段，检测语言: {language}）")
        return " ".join([s["text"] for s in all_segments]), all_segments, language

    except Exception as e:
        logger.log_step("音频转录", "failed", error=str(e))
        raise RuntimeError(f"转录过程失败: {str(e)}")

# ======================
# 会议内容分析（修复dict属性错误）
# ======================
# 分析结果数据模型
class ChunkAnalysis(BaseModel):
    summary: str = Field(description="该片段的总结（100-200字）")
    key_points: list[str] = Field(description="该片段的关键点列表")
    action_items: list[dict] = Field(description="行动项列表，每个包含task、assignee、due_date")
    topics: list[str] = Field(description="讨论的话题列表")

class FullMeetingAnalysis(BaseModel):
    meeting_title: str = Field(description="会议标题")
    participants: list[str] = Field(description="参与者名单")
    summary: str = Field(description="3-5段完整会议总结")
    key_points: dict = Field(description="按话题分组的全局关键点")
    action_items: list[dict] = Field(description="汇总的行动项")
    meeting_type: str = Field(description="会议类型")
    topics_flow: list[str] = Field(description="会议话题流转顺序")

def get_chunk_analysis_chain(language):
    """创建分块分析链（显式传递API密钥）"""
    parser = JsonOutputParser(pydantic_object=ChunkAnalysis)
    prompt = PromptTemplate(
        template="分析以下会议片段，提取关键信息（用{language}）：\n{format_instructions}\n会议片段：{transcript}",
        input_variables=["transcript"],
        partial_variables={
            "language": "中文" if language.startswith('zh') else "English",
            "format_instructions": parser.get_format_instructions()
        }
    )
    # 关键修复：显式传入API密钥
    return prompt | ChatOpenAI(
        model="gpt-3.5-turbo",
        temperature=0.3,
        openai_api_key=OPENAI_API_KEY  # 直接使用全局变量中的密钥
    ) | parser

def get_full_analysis_chain(language):
    """创建全局分析链（显式传递API密钥）"""
    parser = JsonOutputParser(pydantic_object=FullMeetingAnalysis)
    prompt = PromptTemplate(
        template="基于以下各片段分析，生成完整会议报告（用{language}）：\n{format_instructions}\n片段分析：{chunk_analyses}",
        input_variables=["chunk_analyses"],
        partial_variables={
            "language": "中文" if language.startswith('zh') else "English",
            "format_instructions": parser.get_format_instructions()
        }
    )
    # 关键修复：显式传入API密钥
    return prompt | ChatOpenAI(
        model="gpt-4",
        temperature=0.3,
        openai_api_key=OPENAI_API_KEY  # 直接使用全局变量中的密钥
    ) | parser

def analyze_meeting(transcript_segments, language='en'):
    """分析会议内容（分块分析+全局整合）"""
    logger.log_step("会议分析", "started")
    print("\n开始分析会议内容...")

    try:
        # 按时间分割为分析块（45分钟/块）
        ANALYSIS_CHUNK_DURATION = 2700
        analysis_chunks = []
        current_chunk = []

        for seg in transcript_segments:
            if not current_chunk:
                current_chunk.append(seg)
            else:
                if seg["end"] - current_chunk[0]["start"] <= ANALYSIS_CHUNK_DURATION:
                    current_chunk.append(seg)
                else:
                    analysis_chunks.append(current_chunk)
                    current_chunk = [seg]
        if current_chunk:
            analysis_chunks.append(current_chunk)

        print(f"📝 将会议内容分为 {len(analysis_chunks)} 个分析块")

        # 分块分析
        chunk_analyses = []
        chunk_chain = get_chunk_analysis_chain(language)

        for i, chunk in enumerate(tqdm(analysis_chunks, desc="分析进度")):
            # 生成带时间戳的块文本
            chunk_text = "\n".join([
                f"[{str(datetime.timedelta(seconds=int(seg['start'])))}] {seg['text']}"
                for seg in chunk
            ])

            try:
                analysis = chunk_chain.invoke({"transcript": chunk_text[:12000]})  # 限制长度
                chunk_analyses.append({
                    "chunk_id": i,
                    "start_time": chunk[0]["start"],
                    "end_time": chunk[-1]["end"],
                    "analysis": analysis
                })
            except Exception as e:
                print(f"⚠️ 分析块 {i} 失败: {str(e)}")
                continue

        if not chunk_analyses:
            raise RuntimeError("所有分析块处理失败，无法生成报告")

        # 全局整合分析 - 修复：移除.dict()调用，因为分析结果已经是字典
        full_chain = get_full_analysis_chain(language)
        full_analysis = full_chain.invoke({
            "chunk_analyses": json.dumps([
                {
                    "时间段": f"{str(datetime.timedelta(seconds=int(c['start_time'])))} - {str(datetime.timedelta(seconds=int(c['end_time'])))}",
                    "分析": c["analysis"]  # 关键修复：直接使用字典对象
                } for c in chunk_analyses
            ], ensure_ascii=False)
        })

        # 添加元数据 - 修复：如果full_analysis是Pydantic模型，先转字典
        if hasattr(full_analysis, 'dict'):
            full_analysis = full_analysis.dict()

        full_analysis["language"] = language
        full_analysis["date"] = datetime.datetime.now().isoformat()
        full_analysis["total_duration"] = f"{(transcript_segments[-1]['end'] - transcript_segments[0]['start'])/3600:.2f}小时"

        logger.log_step("会议分析", "success")
        print(f"✅ 会议分析完成（{len(full_analysis['topics_flow'])}个话题，{len(full_analysis['action_items'])}个行动项）")
        return full_analysis

    except Exception as e:
        error_msg = f"会议分析失败: {str(e)}"
        logger.log_step("会议分析", "failed", error=error_msg)
        print(f"❌ {error_msg}")
        return {"error": error_msg}

# ======================
# Notion报告生成（修复块数量超限问题）
# ======================
def create_notion_report(meeting_data, transcript_segments):
    """在Notion中创建会议报告（控制块数量≤100）"""
    logger.log_step("创建Notion报告", "started")

    try:
        # 验证父页面是否存在
        try:
            parent_page = notion.pages.retrieve(NOTION_PAGE_ID)
            parent_title = parent_page.get('properties', {}).get('title', {}).get('title', [{}])[0].get('plain_text', '无标题页面')
            print(f"✅ 成功访问父页面: {parent_title}")
        except errors.APIResponseError as e:
            raise RuntimeError(f"Notion父页面访问失败: {str(e)}")

        # 创建新报告页面
        new_page = notion.pages.create(
            parent={"page_id": NOTION_PAGE_ID},
            properties={
                "title": {
                    "title": [{"text": {"content": meeting_data.get("meeting_title", "会议报告")[:200]}}]
                }
            }
        )
        page_id = new_page["id"]

        # 构建报告内容块（控制总数量≤100）
        children_blocks = []

        # 1. 会议概览（约2个块）
        children_blocks.append({
            "object": "block",
            "type": "heading_2",
            "heading_2": {"rich_text": [{"text": {"content": "会议概览"}}]}
        })
        overview_text = f"""
        标题: {meeting_data.get('meeting_title', '未命名会议')}
        日期: {meeting_data.get('date', datetime.datetime.now().strftime('%Y-%m-%d'))}
        总时长: {meeting_data.get('total_duration', '未知')}
        参与者: {', '.join(meeting_data.get('participants', [])) or '未识别'}
        语言: {meeting_data.get('language', '未知')}
        """
        children_blocks.append({
            "object": "block",
            "type": "paragraph",
            "paragraph": {"rich_text": [{"text": {"content": overview_text.strip()}}]}
        })

        # 2. 话题流转（话题数量+1个块）
        children_blocks.append({
            "object": "block",
            "type": "heading_2",
            "heading_2": {"rich_text": [{"text": {"content": "话题流转顺序"}}]}
        })
        # 限制话题数量（避免块过多）
        max_topics = 20  # 最多显示20个话题
        for topic in meeting_data.get('topics_flow', [])[:max_topics]:
            children_blocks.append({
                "object": "block",
                "type": "numbered_list_item",
                "numbered_list_item": {"rich_text": [{"text": {"content": topic}}]}
            })

        # 3. 会议总结（总结段落数+1个块）
        children_blocks.append({
            "object": "block",
            "type": "heading_2",
            "heading_2": {"rich_text": [{"text": {"content": "会议总结"}}]}
        })
        # 限制总结段落数
        max_summary_paras = 5  # 最多5段总结
        for para in meeting_data.get('summary', '').split('\n\n')[:max_summary_paras]:
            if para.strip():
                children_blocks.append({
                    "object": "block",
                    "type": "paragraph",
                    "paragraph": {"rich_text": [{"text": {"content": para.strip()}}]}
                })

        # 4. 关键要点（话题数+要点数+1个块）
        children_blocks.append({
            "object": "block",
            "type": "heading_2",
            "heading_2": {"rich_text": [{"text": {"content": "关键要点"}}]}
        })
        key_points = meeting_data.get('key_points', {})
        max_key_topics = 10  # 最多10个关键话题
        for topic, points in list(key_points.items())[:max_key_topics]:
            children_blocks.append({
                "object": "block",
                "type": "heading_3",
                "heading_3": {"rich_text": [{"text": {"content": topic}}]}
            })
            # 限制每个话题的要点数量
            max_points_per_topic = 5  # 每个话题最多5个要点
            for point in points[:max_points_per_topic]:
                children_blocks.append({
                    "object": "block",
                    "type": "bulleted_list_item",
                    "bulleted_list_item": {"rich_text": [{"text": {"content": point}}]}
                })

        # 5. 行动项表格（行动项数+2个块）
        children_blocks.append({
            "object": "block",
            "type": "heading_2",
            "heading_2": {"rich_text": [{"text": {"content": "行动项"}}]}
        })
        table_rows = []
        max_actions = 20  # 最多显示20个行动项
        for idx, item in enumerate(meeting_data.get('action_items', [])[:max_actions]):
            table_rows.append([
                [{"text": {"content": str(idx+1)}}],
                [{"text": {"content": item.get('task', '')}}],
                [{"text": {"content": item.get('assignee', '未分配')}}],
                [{"text": {"content": item.get('due_date', '无')}}]
            ])
        children_blocks.append({
            "object": "block",
            "type": "table",
            "table": {
                "table_width": 4,
                "has_column_header": True,
                "children": [
                    {
                        "object": "block",
                        "type": "table_row",
                        "table_row": {
                            "cells": [
                                [{"text": {"content": "序号"}}],
                                [{"text": {"content": "任务"}}],
                                [{"text": {"content": "负责人"}}],
                                [{"text": {"content": "截止日期"}}]
                            ]
                        }
                    },
                    *[{
                        "object": "block",
                        "type": "table_row",
                        "table_row": {"cells": cells}
                    } for cells in table_rows]
                ]
            }
        })

        # 6. 转录文本（限制为MAX_TRANSCRIPT_SEGMENTS条，避免块超限）
        children_blocks.append({
            "object": "block",
            "type": "heading_2",
            "heading_2": {"rich_text": [{"text": {"content": f"会议转录文本（节选，共{len(transcript_segments)}条）"}}]}
        })
        # 只显示前MAX_TRANSCRIPT_SEGMENTS条转录文本
        for seg in transcript_segments[:MAX_TRANSCRIPT_SEGMENTS]:
            time_str = str(datetime.timedelta(seconds=int(seg["start"])))
            children_blocks.append({
                "object": "block",
                "type": "paragraph",
                "paragraph": {"rich_text": [{"text": {"content": f"[{time_str}] {seg['text']}"}}]}
            })

        # 检查总块数，确保不超过100
        if len(children_blocks) > 100:
            # 紧急截断（保留核心内容）
            children_blocks = children_blocks[:100]
            print(f"⚠️ 警告：内容块数量超限，已截断为100个块")

        # 将内容添加到页面
        notion.blocks.children.append(block_id=page_id, children=children_blocks)
        report_url = new_page.get("url", "")

        logger.log_step("创建Notion报告", "success", {"报告URL": report_url, "总块数": len(children_blocks)})
        print(f"✅ Notion报告已生成（总块数: {len(children_blocks)}）")
        return report_url

    except Exception as e:
        logger.log_step("创建Notion报告", "failed", error=str(e))
        print(f"❌ 报告生成失败: {str(e)}")
        return None

# ======================
# 输入处理
# ======================
def handle_input():
    """处理用户输入（音频或文本）"""
    print("\n=== 请选择输入方式 ===")
    print("1: 上传音频文件 (.mp3/.wav/.m4a/.opus)")
    print("2: 上传带时间戳的转录文本 (.txt)")

    try:
        choice = input("请选择 (1/2): ").strip() or "1"
    except:
        choice = "1"

    if choice == "1":
        # 处理音频输入
        print("\n请上传音频文件（支持2-3小时会议录音）:")
        uploaded = files.upload()
        if not uploaded:
            print("⚠️ 未检测到上传文件，请重试")
            return handle_input()

        filename = next(iter(uploaded.keys()))
        audio_ext = os.path.splitext(filename)[1].lower()
        supported_ext = ['.mp3', '.wav', '.m4a', '.opus']

        if audio_ext not in supported_ext:
            print(f"❌ 不支持的文件格式（支持: {', '.join(supported_ext)}）")
            return handle_input()

        # 保存音频文件
        audio_path = f"/tmp/{filename}"
        with open(audio_path, 'wb') as f:
            f.write(uploaded[filename])

        # 检查音频时长
        duration = get_audio_duration(audio_path)
        if duration < 3600:  # 小于1小时
            print(f"⚠️ 检测到音频时长较短（{duration/60:.1f}分钟）")
            if input("是否继续使用长会议模式处理? (y/n): ").strip().lower() != 'y':
                os.unlink(audio_path)
                print("已切换到普通模式")
                return handle_input()

        print(f"🎵 音频信息: {filename}（{duration/60:.1f}分钟）")

        # 选择转录模型
        print("\n⚡ 请选择转录模型:")
        print("1: base.en - 快速模式（适合清晰语音）")
        print("2: small.en - 平衡模式（推荐，速度与精度兼顾）")
        print("3: medium.en - 高精度模式（适合复杂会议）")

        model_choice = input("请选择 (1-3，默认2): ").strip() or "2"
        model_map = {"1": "base.en", "2": "small.en", "3": "medium.en"}
        model_size = model_map.get(model_choice, "small.en")
        print(f"将使用 {model_size} 模型进行转录")

        # 执行转录
        try:
            transcript, segments, language = transcribe_long_audio(audio_path, model_size)
            return transcript, segments, language
        except Exception as e:
            if os.path.exists(audio_path):
                os.unlink(audio_path)
            raise

    elif choice == "2":
        # 处理文本输入
        print("\n请上传带时间戳的转录文本（格式示例: [00:05:10] 发言人: ...）:")
        uploaded = files.upload()
        if not uploaded:
            print("⚠️ 未检测到上传文件，请重试")
            return handle_input()

        filename = next(iter(uploaded.keys()))
        if not filename.endswith('.txt'):
            print("❌ 仅支持.txt格式的文本文件")
            return handle_input()

        # 解析文本
        try:
            transcript_text = uploaded[filename].decode('utf-8')
            segments = []
            time_pattern = re.compile(r'\[(\d+:\d+:\d+)\]')  # 匹配[HH:MM:SS]

            for line in transcript_text.split('\n'):
                line = line.strip()
                if not line:
                    continue
                match = time_pattern.search(line)
                if match:
                    time_str = match.group(1)
                    text = time_pattern.sub('', line).strip()
                    # 转换时间为秒
                    h, m, s = map(int, time_str.split(':'))
                    start_time = h * 3600 + m * 60 + s
                    segments.append({
                        "text": text,
                        "start": start_time,
                        "end": start_time + 30  # 估算结束时间
                    })

            if not segments:
                raise ValueError("未检测到有效时间戳，请检查文本格式")

            # 检测语言
            language = detect(transcript_text[:500]) if transcript_text else 'en'
            print(f"✅ 已解析转录文本（{len(segments)}个片段，检测语言: {language}）")
            return transcript_text, segments, language
        except Exception as e:
            print(f"❌ 文本解析失败: {str(e)}")
            raise

    else:
        print("⚠️ 无效选择，默认使用音频输入")
        return handle_input()

# ======================
# 主函数
# ======================
def main():
    """主函数：协调会议处理流程"""
    print("=== 会议分析工具 ===")
    logger.log_step("会议处理流程", "started")

    try:
        # 验证GPU环境
        if not torch.cuda.is_available():
            raise RuntimeError("请切换到GPU环境（Runtime > Change runtime type > 选择GPU）")
        print(f"✅ 检测到GPU: {torch.cuda.get_device_name(0)}")

        # 处理输入
        transcript, segments, language = handle_input()
        if not transcript or not segments:
            raise RuntimeError("未获取到有效会议内容")

        # 分析会议
        meeting_data = analyze_meeting(segments, language)
        if "error" in meeting_data:
            raise RuntimeError(meeting_data["error"])

        # 生成Notion报告
        report_url = create_notion_report(meeting_data, segments)
        if not report_url:
            raise RuntimeError("无法生成Notion报告")

        # 保存日志并输出结果
        log_file = logger.save_logs()
        print(f"\n🎉 会议处理完成！")
        print(f"📄 会议报告: {report_url}")
        print(f"📋 处理日志已保存到: {log_file}")

        # 显示报告链接
        from IPython.display import HTML
        display(HTML(f'<a href="{report_url}" target="_blank">点击查看Notion会议报告</a>'))

    except Exception as e:
        logger.save_logs("meeting_error_logs.json")
        print(f"\n❌ 处理失败: {str(e)}")
        print("错误详情已保存到 meeting_error_logs.json")

if __name__ == "__main__":
    main()

In [ ]:
# 安装必要依赖（指定兼容版本）
!pip uninstall -y whisper
!pip install faster-whisper==0.10.0  # 锁定版本以避免API变更
!pip install git+https://github.com/openai/whisper.git
!pip install tqdm python-docx notion-client langdetect langchain==0.1.13 langchain-openai==0.0.8 pydantic==2.5.2 httpx==0.27.0
!sudo apt update && sudo apt install ffmpeg -y

import os
import json
import re
import torch
import subprocess
import datetime
import tempfile
from tqdm import tqdm
from pydantic import BaseModel, Field
import httpx
import concurrent.futures
from functools import partial

# 导入第三方库
from google.colab import files, userdata
from notion_client import Client, errors
from langdetect import detect, LangDetectException
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser

# 清除代理环境变量（避免网络连接问题）
for var in ['HTTP_PROXY', 'HTTPS_PROXY', 'http_proxy', 'https_proxy']:
    if var in os.environ:
        del os.environ[var]

# 初始化Notion客户端（修复httpx代理参数问题）
http_client = httpx.Client()
http_client.proxies = None  # 禁用代理

notion = Client(
    auth=userdata.get('NOTION_TOKEN'),
    client=http_client
)

# ======================
# 配置参数与初始化
# ======================
try:
    # 从环境变量获取密钥
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    NOTION_TOKEN = userdata.get('NOTION_TOKEN')
    NOTION_PAGE_ID = userdata.get('NOTION_PAGE_ID')
    NOTION_DB_ID = userdata.get('NOTION_DB_ID')  # 数据库ID

    # 长音频处理参数
    CHUNK_DURATION = 900  # 每块15分钟（秒）
    OVERLAP_DURATION = 30  # 块间重叠30秒
    MAX_CONCURRENT_CHUNKS = 1  # 单线程处理，避免文件竞争
    # Notion块数量限制相关参数
    MAX_TRANSCRIPT_SEGMENTS = 50  # 最多显示50条转录文本
    NOTION_RICH_TEXT_LIMIT = 1950  # Notion rich_text字段最大长度（留50字符余量）

    # 验证必要密钥
    missing_creds = []
    if not OPENAI_API_KEY:
        missing_creds.append("OPENAI_API_KEY")
    if not NOTION_TOKEN:
        missing_creds.append("NOTION_TOKEN")
    if not NOTION_PAGE_ID:
        missing_creds.append("NOTION_PAGE_ID")
    if not NOTION_DB_ID:
        missing_creds.append("NOTION_DB_ID")

    if missing_creds:
        raise ValueError(f"缺少必要凭证: {', '.join(missing_creds)}")

    print("✅ 所有凭证已配置完成，准备处理会议内容")

except Exception as e:
    print(f"❌ 初始化失败: {str(e)}")
    print("\n设置指南:")
    print("1. 点击左侧边栏的钥匙图标（Colab Secrets）")
    print("2. 添加以下密钥:")
    print("   - OPENAI_API_KEY: 你的OpenAI API密钥")
    print("   - NOTION_TOKEN: Notion集成令牌")
    print("   - NOTION_PAGE_ID: 会议集成页ID")
    print("   - NOTION_DB_ID: 目标数据库ID")
    raise

# ======================
# 日志记录系统
# ======================
class MeetingLogger:
    def __init__(self):
        self.logs = {
            "start_time": datetime.datetime.now().isoformat(),
            "steps": [],
            "chunk_status": {}
        }

    def log_step(self, step_name, status, details=None, error=None):
        entry = {
            "step": step_name,
            "timestamp": datetime.datetime.now().isoformat(),
            "status": status
        }
        if details:
            entry["details"] = details
        if error:
            entry["error"] = str(error)
        self.logs["steps"].append(entry)

    def log_chunk(self, chunk_id, status, error=None):
        self.logs["chunk_status"][chunk_id] = {
            "status": status,
            "timestamp": datetime.datetime.now().isoformat(),
            "error": str(error) if error else None
        }

    def get_completed_chunks(self):
        return [k for k, v in self.logs["chunk_status"].items() if v["status"] == "success"]

    def get_failed_chunks(self):
        return [k for k, v in self.logs["chunk_status"].items() if v["status"] == "failed"]

    def save_logs(self, filename="meeting_logs.json"):
        with open(filename, "w") as f:
            json.dump(self.logs, f, indent=2)
        return filename

logger = MeetingLogger()

# ======================
# 音频处理工具
# ======================
def get_audio_duration(audio_path):
    try:
        result = subprocess.run(
            ["ffmpeg", "-i", audio_path],
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True
        )
        output = result.stdout

        duration_match = re.search(r"Duration: (\d+:\d+:\d+\.\d+)", output)
        if not duration_match:
            return 0.0

        duration_str = duration_match.group(1)
        h, m, s = duration_str.split(':')
        return float(h) * 3600 + float(m) * 60 + float(s)

    except Exception as e:
        logger.log_step("获取音频时长", "warning", error=str(e))
        return 0.0

def split_audio_into_chunks(audio_path):
    logger.log_step("音频分块", "started")

    try:
        total_duration = get_audio_duration(audio_path)
        if total_duration <= 0:
            raise ValueError("无法获取有效音频时长，可能文件损坏")

        num_chunks = max(1, int((total_duration + CHUNK_DURATION - OVERLAP_DURATION) //
                              (CHUNK_DURATION - OVERLAP_DURATION)))
        logger.log_step("计算分块数量", "success", {"总时长(分钟)": f"{total_duration/60:.1f}", "分块数": num_chunks})
        print(f"📊 音频将分割为 {num_chunks} 块（每块15分钟，重叠30秒）")

        chunk_dir = tempfile.mkdtemp()
        chunks = []

        for i in range(num_chunks):
            start_time = i * (CHUNK_DURATION - OVERLAP_DURATION)
            end_time = min(start_time + CHUNK_DURATION, total_duration)

            start_str = str(datetime.timedelta(seconds=start_time))
            duration_str = str(datetime.timedelta(seconds=end_time - start_time))

            chunk_path = f"{chunk_dir}/chunk_{i:03d}.wav"

            subprocess.run(
                [
                    "ffmpeg", "-y",
                    "-i", audio_path,
                    "-ss", start_str,
                    "-t", duration_str,
                    "-ar", "16000",
                    "-ac", "1",
                    "-acodec", "pcm_s16le",
                    chunk_path
                ],
                check=True,
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE
            )

            if not os.path.exists(chunk_path) or os.path.getsize(chunk_path) < 1024:
                raise RuntimeError(f"分块 {i} 生成失败，文件大小异常")

            chunks.append({
                "id": i,
                "path": chunk_path,
                "start_time": start_time,
                "end_time": end_time,
                "dir": chunk_dir
            })

        logger.log_step("音频分块", "success")
        return chunks

    except Exception as e:
        logger.log_step("音频分块", "failed", error=str(e))
        raise RuntimeError(f"音频分块失败: {str(e)}")

# ======================
# 语音转录
# ======================
global_model = None

def transcribe_chunk(chunk):
    chunk_id = chunk["id"]
    chunk_path = chunk["path"]
    start_time = chunk["start_time"]

    try:
        from faster_whisper import WhisperModel
        global global_model

        if global_model is None:
            if not torch.cuda.is_available():
                raise RuntimeError("请切换到GPU环境（Runtime > Change runtime type）")

            print(f"🔧 加载 {chunk['model_size']} 模型（首次运行需下载约1.5GB）...")
            global_model = WhisperModel(
                chunk["model_size"],
                device="cuda",
                compute_type="float16",
                download_root="/content/models"
            )
            print(f"✅ 模型加载成功")

        if not os.path.exists(chunk_path):
            raise FileNotFoundError(f"分块文件不存在: {chunk_path}")

        segments, info = global_model.transcribe(
            chunk_path,
            beam_size=2,
            vad_filter=True,
            vad_parameters=dict(min_silence_duration_ms=300),
            language=None  # 自动检测语言
        )

        timestamped_segments = []
        for seg in segments:
            timestamped_segments.append({
                "text": seg.text.strip(),
                "start": seg.start + start_time,
                "end": seg.end + start_time,
                "language": info.language  # 记录当前块的语言
            })

        logger.log_chunk(chunk_id, "success")
        return {
            "chunk_id": chunk_id,
            "segments": timestamped_segments,
            "language": info.language
        }

    except Exception as e:
        logger.log_chunk(chunk_id, "failed", error=str(e))
        print(f"⚠️ 分块 {chunk_id} 转录失败: {str(e)}")
        return None

def transcribe_long_audio(audio_path, model_size="small.en"):
    logger.log_step("音频转录", "started", {"模型": model_size})

    try:
        global global_model
        global_model = None

        chunks = split_audio_into_chunks(audio_path)
        num_chunks = len(chunks)

        for chunk in chunks:
            chunk["model_size"] = model_size

        print(f"🚀 开始转录 {num_chunks} 个分块...")
        results = []
        for chunk in tqdm(chunks, desc="转录进度"):
            result = transcribe_chunk(chunk)
            if result:
                results.append(result)

        failed_ids = logger.get_failed_chunks()
        if failed_ids:
            print(f"🔄 重试 {len(failed_ids)} 个失败分块...")
            for chunk in chunks:
                if chunk["id"] in failed_ids:
                    result = transcribe_chunk(chunk)
                    if result:
                        results.append(result)

        results.sort(key=lambda x: x["chunk_id"])
        all_segments = []
        for res in results:
            all_segments.extend(res["segments"])

        unique_dirs = list({chunk["dir"] for chunk in chunks})
        for dir_path in unique_dirs:
            if os.path.exists(dir_path):
                for f in os.listdir(dir_path):
                    os.unlink(os.path.join(dir_path, f))
                os.rmdir(dir_path)

        # 提取所有出现的语言（去重）
        all_languages = list({seg.get("language", "unknown") for seg in all_segments})
        primary_language = results[0]["language"] if results else "en"

        # 计算会议总时长（秒）
        total_seconds = all_segments[-1]["end"] - all_segments[0]["start"] if all_segments else 0

        logger.log_step("音频转录", "success", {"总片段数": len(all_segments), "所有语言": all_languages})
        print(f"✅ 转录完成（{len(all_segments)}个片段，主要语言: {primary_language}，所有语言: {all_languages}）")
        return " ".join([s["text"] for s in all_segments]), all_segments, primary_language, all_languages, total_seconds

    except Exception as e:
        logger.log_step("音频转录", "failed", error=str(e))
        raise RuntimeError(f"转录过程失败: {str(e)}")

# ======================
# 会议内容分析
# ======================
class ChunkAnalysis(BaseModel):
    summary: str = Field(description="该片段的总结（100-200字）")
    key_points: list[str] = Field(description="该片段的关键点键列表（客观事实）")
    action_items: list[dict] = Field(description="行动项列表，每个包含task、assignee、due_date")
    topics: list[str] = Field(description="讨论的话题列表")
    decisions: list[str] = Field(description="该片段中达成的具体决定（明确的结论）")
    concerns: list[str] = Field(description="该片段中提出的担忧、问题或风险")
    platform: str = Field(description="会议发生的平台或场所（如Zoom、会议室A等）")  # 新增字段

class FullMeetingAnalysis(BaseModel):
    meeting_title: str = Field(description="会议标题")
    participants: list[str] = Field(description="参与者名单")
    summary: str = Field(description="3-5段完整会议总结")
    key_points: dict = Field(description="按话题分组组的全局关键点（客观事实）")
    action_items: list[dict] = Field(description="汇总的行动项")
    meeting_type: str = Field(description="会议类型（如周会、项目评审会、头脑风暴等）")
    topics_flow: list[str] = Field(description="会议话题流转顺序")
    decisions: list[str] = Field(description="会议中达成的所有决定（明确结论，如“同意项目延期”）")
    concerns: list[str] = Field(description="会议中提出的所有担忧、问题或风险（如“资源不足”）")
    platform: str = Field(description="会议发生的平台或场所（如Zoom、Teams、会议室B等）")  # 新增字段

def get_chunk_analysis_chain(language):
    parser = JsonOutputParser(pydantic_object=ChunkAnalysis)
    prompt = PromptTemplate(
        template="""分析以下会议片段，提取关键信息（用{language}）：
{format_instructions}
注意：
- key_points：客观事实（如“项目进度落后20%”）
- decisions：明确达成的结论（如“决定增加2名开发人员”）
- concerns：提出的担忧（如“预算可能超支”）
- platform：会议进行的平台或物理场所（如Zoom、公司3楼会议室等）
会议片段：{transcript}""",
        input_variables=["transcript"],
        partial_variables={
            "language": "中文" if language.startswith('zh') else "English",
            "format_instructions": parser.get_format_instructions()
        }
    )
    return prompt | ChatOpenAI(
        model="gpt-3.5-turbo",
        temperature=0.3,
        openai_api_key=OPENAI_API_KEY
    ) | parser

def get_full_analysis_chain(language):
    parser = JsonOutputParser(pydantic_object=FullMeetingAnalysis)
    prompt = PromptTemplate(
        template="""基于以下各片段分析，生成完整会议报告（用{language}）：
{format_instructions}
注意：
- key_points：仅包含客观事实，不包含结论
- decisions：必须是明确达成的结论（有具体结果）
- concerns：必须是提出的问题或风险（未解决的担忧）
- platform：明确会议发生的平台或场所（如Zoom、Teams、总部会议室等）
- meeting_type：明确会议类型（如周例会、项目启动会、评审会等）
片段分析：{chunk_analyses}""",
        input_variables=["chunk_analyses"],
        partial_variables={
            "language": "中文" if language.startswith('zh') else "English",
            "format_instructions": parser.get_format_instructions()
        }
    )
    return prompt | ChatOpenAI(
        model="gpt-4",
        temperature=0.3,
        openai_api_key=OPENAI_API_KEY
    ) | parser

def analyze_meeting(transcript_segments, language='en'):
    logger.log_step("会议分析", "started")
    print("\n开始分析会议内容...")

    try:
        ANALYSIS_CHUNK_DURATION = 2700
        analysis_chunks = []
        current_chunk = []

        for seg in transcript_segments:
            if not current_chunk:
                current_chunk.append(seg)
            else:
                if seg["end"] - current_chunk[0]["start"] <= ANALYSIS_CHUNK_DURATION:
                    current_chunk.append(seg)
                else:
                    analysis_chunks.append(current_chunk)
                    current_chunk = [seg]
        if current_chunk:
            analysis_chunks.append(current_chunk)

        print(f"📝 将会议内容分为 {len(analysis_chunks)} 个分析块")

        chunk_analyses = []
        chunk_chain = get_chunk_analysis_chain(language)

        for i, chunk in enumerate(tqdm(analysis_chunks, desc="分析进度")):
            chunk_text = "\n".join([
                f"[{str(datetime.timedelta(seconds=int(seg['start'])))}] {seg['text']}"
                for seg in chunk
            ])

            try:
                analysis = chunk_chain.invoke({"transcript": chunk_text[:12000]})
                chunk_analyses.append({
                    "chunk_id": i,
                    "start_time": chunk[0]["start"],
                    "end_time": chunk[-1]["end"],
                    "analysis": analysis
                })
            except Exception as e:
                print(f"⚠️ 分析块 {i} 失败: {str(e)}")
                continue

        if not chunk_analyses:
            raise RuntimeError("所有分析块处理失败，无法生成报告")

        full_chain = get_full_analysis_chain(language)
        full_analysis = full_chain.invoke({
            "chunk_analyses": json.dumps([
                {
                    "时间段": f"{str(datetime.timedelta(seconds=int(c['start_time'])))} - {str(datetime.timedelta(seconds=int(c['end_time'])))}",
                    "分析": c["analysis"]
                } for c in chunk_analyses
            ], ensure_ascii=False)
        })

        if hasattr(full_analysis, 'dict'):
            full_analysis = full_analysis.dict()

        full_analysis["language"] = language
        full_analysis["date"] = datetime.datetime.now().isoformat()

        logger.log_step("会议分析", "success")
        print(f"✅ 会议分析完成（{len(full_analysis.get('topics_flow', []))}个话题，{len(full_analysis.get('action_items', []))}个行动项）")
        return full_analysis

    except Exception as e:
        error_msg = f"会议分析失败: {str(e)}"
        logger.log_step("会议分析", "failed", error=error_msg)
        print(f"❌ {error_msg}")
        return {"error": error_msg}

# ======================
# Notion操作
# ======================
def create_notion_report(meeting_data, transcript_segments):
    logger.log_step("创建Notion报告", "started")

    try:
        try:
            parent_page = notion.pages.retrieve(NOTION_PAGE_ID)
            parent_title = parent_page.get('properties', {}).get('title', {}).get('title', [{}])[0].get('plain_text', '无标题页面')
            print(f"✅ 成功访问父页面: {parent_title}")
        except errors.APIResponseError as e:
            raise RuntimeError(f"Notion父页面访问失败: {str(e)}")

        new_page = notion.pages.create(
            parent={"page_id": NOTION_PAGE_ID},
            properties={
                "title": {
                    "title": [{"text": {"content": meeting_data.get("meeting_title", "会议报告")[:200]}}]
                }
            }
        )
        page_id = new_page["id"]

        # 构建报告内容块
        children_blocks = []

        # 1. 会议概览
        children_blocks.append({
            "object": "block",
            "type": "heading_2",
            "heading_2": {"rich_text": [{"text": {"content": "会议概览"}}]}
        })
        overview_text = f"""
        **标题**: {meeting_data.get('meeting_title', '未命名会议')}
        **日期**: {meeting_data.get('date', datetime.datetime.now().strftime('%Y年%m月%d日'))}
        **参与者**: {', '.join(meeting_data.get('participants', [])) or '未识别'}
        **类型**: {meeting_data.get('meeting_type', '未指定')}
        **平台**: {meeting_data.get('platform', '未指定')}
        **语言**: {', '.join(meeting_data.get('all_languages', [])) or '未知'}
        """
        children_blocks.append({
            "object": "block",
            "type": "paragraph",
            "paragraph": {"rich_text": [{"text": {"content": overview_text.strip()}}]}
        })

        # 2. 话题流转
        children_blocks.append({
            "object": "block",
            "type": "heading_2",
            "heading_2": {"rich_text": [{"text": {"content": "话题流转顺序"}}]}
        })
        max_topics = 20
        for topic in meeting_data.get('topics_flow', [])[:max_topics]:
            children_blocks.append({
                "object": "block",
                "type": "numbered_list_item",
                "numbered_list_item": {"rich_text": [{"text": {"content": topic}}]}
            })

        # 3. 会议总结
        children_blocks.append({
            "object": "block",
            "type": "heading_2",
            "heading_2": {"rich_text": [{"text": {"content": "会议总结"}}]}
        })
        max_summary_paras = 5
        for para in meeting_data.get('summary', '').split('\n\n')[:max_summary_paras]:
            if para.strip():
                children_blocks.append({
                    "object": "block",
                    "type": "paragraph",
                    "paragraph": {"rich_text": [{"text": {"content": para.strip()}}]}
                })

        # 4. 关键要点
        children_blocks.append({
            "object": "block",
            "type": "heading_2",
            "heading_2": {"rich_text": [{"text": {"content": "关键要点"}}]}
        })
        key_points = meeting_data.get('key_points', {})
        max_key_topics = 10
        for topic, points in list(key_points.items())[:max_key_topics]:
            children_blocks.append({
                "object": "block",
                "type": "heading_3",
                "heading_3": {"rich_text": [{"text": {"content": topic}}]}
            })
            max_points_per_topic = 5
            for point in points[:max_points_per_topic]:
                children_blocks.append({
                    "object": "block",
                    "type": "bulleted_list_item",
                    "bulleted_list_item": {"rich_text": [{"text": {"content": point}}]}
                })

        # 5. 行动项表格
        children_blocks.append({
            "object": "block",
            "type": "heading_2",
            "heading_2": {"rich_text": [{"text": {"content": "行动项"}}]}
        })
        table_rows = []
        max_actions = 20
        for idx, item in enumerate(meeting_data.get('action_items', [])[:max_actions]):
            table_rows.append([
                [{"text": {"content": str(idx+1)}}],
                [{"text": {"content": item.get('task', '')}}],
                [{"text": {"content": item.get('assignee', '未分配')}}],
                [{"text": {"content": item.get('due_date', '无')}}]
            ])
        children_blocks.append({
            "object": "block",
            "type": "table",
            "table": {
                "table_width": 4,
                "has_column_header": True,
                "children": [
                    {
                        "object": "block",
                        "type": "table_row",
                        "table_row": {
                            "cells": [
                                [{"text": {"content": "序号"}}],
                                [{"text": {"content": "任务"}}],
                                [{"text": {"content": "负责人"}}],
                                [{"text": {"content": "截止日期"}}]
                            ]
                        }
                    },
                    *[{
                        "object": "block",
                        "type": "table_row",
                        "table_row": {"cells": cells}
                    } for cells in table_rows]
                ]
            }
        })

        # 6. 转录文本
        children_blocks.append({
            "object": "block",
            "type": "heading_2",
            "heading_2": {"rich_text": [{"text": {"content": f"会议转录文本（节选，共{len(transcript_segments)}条）"}}]}
        })
        for seg in transcript_segments[:MAX_TRANSCRIPT_SEGMENTS]:
            time_str = str(datetime.timedelta(seconds=int(seg["start"])))
            children_blocks.append({
                "object": "block",
                "type": "paragraph",
                "paragraph": {"rich_text": [{"text": {"content": f"[{time_str}] {seg['text']}"}}]}
            })

        # 检查总块数
        if len(children_blocks) > 100:
            children_blocks = children_blocks[:100]
            print(f"⚠️ 警告：内容块数量超限，已截断为100个块")

        notion.blocks.children.append(block_id=page_id, children=children_blocks)
        report_url = new_page.get("url", "")

        logger.log_step("创建Notion报告", "success", {"报告URL": report_url, "总块数": len(children_blocks)})
        print(f"✅ Notion报告已生成（总块数: {len(children_blocks)}）")
        return report_url

    except Exception as e:
        logger.log_step("创建Notion报告", "failed", error=str(e))
        print(f"❌ 报告生成失败: {str(e)}")
        return None

# 写入Notion数据库（核心修改：匹配新表头）
def write_to_notion_database(meeting_data, full_transcript, all_languages, total_seconds):
    logger.log_step("写入Notion数据库", "started")

    try:
        # 辅助函数：截断文本到Notion允许的最大长度
        def truncate_text(text, max_length):
            if text and len(text) > max_length:
                return text[:max_length-3] + "..."  # 预留3个字符给省略号
            return text or ""

        # 辅助函数：转换秒数为"x小时x分钟"格式（不足1分钟按1分钟算）
        def format_duration(seconds):
            hours = seconds // 3600
            remaining_seconds = seconds % 3600
            minutes = (remaining_seconds + 59) // 60  # 向上取整
            return f"{hours}小时{minutes}分钟"

        # 辅助函数：转换秒数为总分钟数（用于数字类型的Duration字段）
        def get_total_minutes(seconds):
            return (seconds + 59) // 60  # 向上取整

        # 1. 准备数据库字段内容（严格匹配表头）
        database_properties = {
            # 会议标题（文本类型）
            "Meeting Title": {"title": [{"text": {"content": meeting_data.get("meeting_title", "Untitled")}}]},
            # 参与者（文本类型）
            "Participant": {
                "rich_text": [{"text": {"content": truncate_text(
                    ", ".join(meeting_data.get("participant", [])),
                    NOTION_RICH_TEXT_LIMIT
                )}}]
            },
            # 日期（日期类型，格式xxxx年xx月xx日）
            "Date": {
                "date": {
                    "start": datetime.datetime.now().strftime("%Y-%m-%d"),  # Notion日期格式需为ISO
                    "end": None
                }
            },
            # 时长（数字类型，存储总分钟数）
            "Duration": {
                "number": get_total_minutes(total_seconds)
            },
            # 会议类型（文本类型）
            "Meeting Type": {
                "rich_text": [{"text": {"content": truncate_text(
                    meeting_data.get("meeting_type", "未指定"),
                    NOTION_RICH_TEXT_LIMIT
                )}}]
            },
            # 会议平台（选择类型，确保值在预设选项中）
            "Platform": {"select": {"name": meeting_data.get("platform", "Unknown")}},
            # 会议完整原文（文本类型）
            "original meeting script": {
                "rich_text": [{"text": {"content": truncate_text(
                    full_transcript,
                    NOTION_RICH_TEXT_LIMIT
                )}}]
            },
            # 会议语言（文本类型）
            "language": {
                "rich_text": [{"text": {"content": truncate_text(
                    ", ".join(all_languages),
                    NOTION_RICH_TEXT_LIMIT
                )}}]
            },
            # 会议总结（文本类型）
            "Summary": {
                "rich_text": [{"text": {"content": truncate_text(
                    meeting_data.get("summary", ""),
                    NOTION_RICH_TEXT_LIMIT
                )}}]
            },
            # 会议决定（文本类型）
            "Decisions": {
                "rich_text": [{"text": {"content": truncate_text(
                    "\n".join([f"- {d}" for d in meeting_data.get("decisions", [])]),
                    NOTION_RICH_TEXT_LIMIT
                )}}]
            },
            # 会议担忧（文本类型）
            "Concerns": {
                "rich_text": [{"text": {"content": truncate_text(
                    "\n".join([f"- {c}" for c in meeting_data.get("concerns", [])]),
                    NOTION_RICH_TEXT_LIMIT
                )}}]
            },
            # 会议要点（文本类型）
            "Key Points": {
                "rich_text": [{"text": {"content": truncate_text(
                    "\n".join([f"- {topic}: {', '.join(points)}"
                              for topic, points in meeting_data.get("key_points", {}).items()]),
                    NOTION_RICH_TEXT_LIMIT
                )}}]
            },
            # 行动项（文本类型）
            "Action Items": {
                "rich_text": [{"text": {"content": truncate_text(
                    "\n".join([f"- {item.get('task', '')}（负责人：{item.get('assignee', '未分配')}）"
                              for item in meeting_data.get("action_items", [])]),
                    NOTION_RICH_TEXT_LIMIT
                )}}]
            }
        }

        # 2. 写入数据库
        new_db_page = notion.pages.create(
            parent={"database_id": NOTION_DB_ID},
            properties=database_properties
        )

        logger.log_step("写入Notion数据库", "success", {"数据库页面ID": new_db_page["id"]})
        print(f"✅ 成功写入Notion数据库（条目ID: {new_db_page['id']}）")
        print(f"⏱️ 会议时长: {format_duration(total_seconds)}")
        return new_db_page["id"]

    except errors.APIResponseError as e:
        error_msg = f"数据库写入失败: {str(e)}"
        logger.log_step("写入Notion数据库", "failed", error=error_msg)
        print(f"❌ {error_msg}（请检查数据库表头名称和类型是否与代码一致）")
        return None
    except Exception as e:
        error_msg = f"数据库写入失败: {str(e)}"
        logger.log_step("写入Notion数据库", "failed", error=error_msg)
        print(f"❌ {error_msg}")
        return None

# ======================
# 输入处理
# ======================
def handle_input():
    print("\n=== 请选择输入方式 ===")
    print("1: 上传音频文件 (.mp3/.wav/.m4a/.opus)")
    print("2: 上传带时间戳的转录文本 (.txt)")

    try:
        choice = input("请选择 (1/2): ").strip() or "1"
    except:
        choice = "1"

    if choice == "1":
        print("\n请上传音频文件（支持2-3小时会议录音）:")
        uploaded = files.upload()
        if not uploaded:
            print("⚠️ 未检测到上传文件，请重试")
            return handle_input()

        filename = next(iter(uploaded.keys()))
        audio_ext = os.path.splitext(filename)[1].lower()
        supported_ext = ['.mp3', '.wav', '.m4a', '.opus']

        if audio_ext not in supported_ext:
            print(f"❌ 不支持的文件格式（支持: {', '.join(supported_ext)}）")
            return handle_input()

        audio_path = f"/tmp/{filename}"
        with open(audio_path, 'wb') as f:
            f.write(uploaded[filename])

        duration = get_audio_duration(audio_path)
        if duration < 3600:
            print(f"⚠️ 检测到音频时长较短（{duration/60:.1f}分钟）")
            if input("是否继续使用长会议模式处理? (y/n): ").strip().lower() != 'y':
                os.unlink(audio_path)
                print("已切换到普通模式")
                return handle_input()

        print(f"🎵 音频信息: {filename}（{duration/60:.1f}分钟）")

        print("\n⚡ 请选择转录模型:")
        print("1: base.en - 快速模式（适合清晰语音）")
        print("2: small.en - 平衡模式（推荐，速度与精度兼顾）")
        print("3: medium.en - 高精度模式（适合复杂会议）")

        model_choice = input("请选择 (1-3，默认2): ").strip() or "2"
        model_map = {"1": "base.en", "2": "small.en", "3": "medium.en"}
        model_size = model_map.get(model_choice, "small.en")
        print(f"将使用 {model_size} 模型进行转录")

        # 新增返回总时长（秒）
        transcript, segments, primary_language, all_languages, total_seconds = transcribe_long_audio(audio_path, model_size)
        return transcript, segments, primary_language, all_languages, total_seconds

    elif choice == "2":
        print("\n请上传带时间戳的转录文本（格式示例: [00:05:10] 发言人: ...）:")
        uploaded = files.upload()
        if not uploaded:
            print("⚠️ 未检测到上传文件，请重试")
            return handle_input()

        filename = next(iter(uploaded.keys()))
        if not filename.endswith('.txt'):
            print("❌ 仅支持.txt格式的文本文件")
            return handle_input()

        try:
            transcript_text = uploaded[filename].decode('utf-8')
            segments = []
            time_pattern = re.compile(r'\[(\d+:\d+:\d+)\]')

            for line in transcript_text.split('\n'):
                line = line.strip()
                if not line:
                    continue
                match = time_pattern.search(line)
                if match:
                    time_str = match.group(1)
                    text = time_pattern.sub('', line).strip()
                    h, m, s = map(int, time_str.split(':'))
                    start_time = h * 3600 + m * 60 + s
                    segments.append({
                        "text": text,
                        "start": start_time,
                        "end": start_time + 30,
                        "language": detect(text) if text.strip() else "en"
                    })

            if not segments:
                raise ValueError("未检测到有效时间戳，请检查文本格式")

            # 计算总时长（秒）
            total_seconds = segments[-1]["end"] - segments[0]["start"] if segments else 0

            all_languages = list({seg["language"] for seg in segments})
            primary_language = all_languages[0] if all_languages else "en"
            print(f"✅ 已解析转录文本（{len(segments)}个片段，检测语言: {all_languages}）")
            return transcript_text, segments, primary_language, all_languages, total_seconds

        except Exception as e:
            print(f"❌ 文本解析失败: {str(e)}")
            raise

    else:
        print("⚠️ 无效选择，默认使用音频输入")
        return handle_input()

# ======================
# 主函数
# ======================
def main():
    print("=== 会议分析工具 ===")
    logger.log_step("会议处理流程", "started")

    try:
        if not torch.cuda.is_available():
            raise RuntimeError("请切换到GPU环境，请切换到GPU环境（Runtime > Change runtime type > 选择GPU）")
        print(f"✅ 检测到GPU: {torch.cuda.get_device_name(0)}")

        # 处理输入（获取总时长）
        transcript, segments, language, all_languages, total_seconds = handle_input()
        if not transcript or not segments:
            raise RuntimeError("未获取到有效会议内容")

        # 分析会议
        meeting_data = analyze_meeting(segments, language)
        if "error" in meeting_data:
            raise RuntimeError(meeting_data["error"])

        # 补充语言信息到会议数据
        meeting_data["all_languages"] = all_languages

        # 生成Notion报告
        report_url = create_notion_report(meeting_data, segments)
        if not report_url:
            raise RuntimeError("无法生成Notion报告")

        # 写入数据库
        db_entry_id = write_to_notion_database(meeting_data, transcript, all_languages, total_seconds)
        if not db_entry_id:
            raise RuntimeError("数据库写入失败，但报告已生成")

        # 保存日志并输出结果
        log_file = logger.save_logs()
        print(f"\n🎉 会议处理完成！")
        print(f"📄 会议报告: {report_url}")
        print(f"📊 数据库条目ID: {db_entry_id}")
        print(f"📋 处理日志已保存到: {log_file}")

        from IPython.display import HTML
        display(HTML(f'<a href="{report_url}" target="_blank">点击查看Notion会议报告</a>'))

    except Exception as e:
        logger.save_logs("meeting_error_logs.json")
        print(f"\n❌ 处理失败: {str(e)}")
        print("错误详情已保存到 meeting_error_logs.json")

if __name__ == "__main__":
    main()
